# Novojob.com

In [2]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

def scrap_novojob():
    # Liste des liens pour chaque catégorie
    categories = [
        "toutes les offres d'emploi",
        "juridique,fiscal,audit,conseil",
        "administrations,moyens généraux",
        "assistanat,secrétariat",
        "metiers banque et assurances",
        "RH,personnel,formation",
        "education,enseignement",
        "direction générale,direction d'unité",
        "vente,televente,assistanat",
        "commercial,technico commercial,service client",
        "responsable commercial,grands comptes",
        "créatio, design",
        "marketing, communication",
        "journalisme,médias,traduction",
        "informatique,systèmes d'information,internet",
        "télécommunication,réseaux",
        "chantier,métiers BTP,architecture",
        "ingénierie,etudes,projet,R&D",
        "logistique,achat,stock,transport",
        "production,méthode,industrie",
        "maintenance,entretien",
        "Qualité,sécurité,Environnement",
        "Santé,Médical,Pharmacie",
        "Hotellerie,Tourisme,Restauration, Loisirs",
        "Ouvriers qualifiés, Chauffeur",
        "autre",
        "Métiers de l'agriculture"
    ]

    base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
    category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

    # Utilisation d'un en-tête pour éviter d'être bloqué
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    intitules_list = []
    entreprises_list = []
    pays_list = []
    dates_list = []
    niveau_list = []
    experience_list = []

    url_list = []
    all_job_lien = []

    # Parcourir les liens de chaque catégorie
    for category_link in category_links:
        req = requests.get(category_link, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

        offres = soup.find_all('div', class_='row-fluid job-details pointer')

        for offre in offres:
            offre_link_tag = offre.find('a')
            if offre_link_tag:
                offre_link = offre_link_tag['href']
                all_job_lien.append(offre_link)

            entreprise_tag = offre.find('h6', class_='ellipsis')
            entreprise = entreprise_tag.get_text().strip() if entreprise_tag else None

            intitule_tag = offre.find('h2', class_='ellipsis row-fluid')
            intitule = intitule_tag.get_text().strip() if intitule_tag else None

            bloc_bottom = offre.find_next('div', class_='bloc-bottom')

            pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
            pays = pays_info.find_parent().text.strip() if pays_info else None

            date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
            date = date_info.find_parent().text.strip() if date_info else None

            niveau_info = bloc_bottom.find('i', class_='fa fa-bookmark icon-left')
            niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

            match = re.match(r'(.+) \((.+)\)', niveau_text)
            niveau_col, experience_col = match.groups() if match else (None, None)

            intitules_list.append(intitule)
            entreprises_list.append(entreprise)
            pays_list.append(pays)
            dates_list.append(date)
            niveau_list.append(niveau_col)
            experience_list.append(experience_col)
            
            url_list.append(category_link)

    # Création du DataFrame
    df_offers = pd.DataFrame({
        'Intitule': intitules_list,
        'Entreprise': entreprises_list,
        'Pays': pays_list,
        'Date': dates_list,
        'Niveau': niveau_list,
        'Experience_lettre': experience_list,
        'url': url_list,
        'Offre_Link': all_job_lien
    })

    # Importer les détails de chaque offre d'emploi
    job_urls = list(df_offers['Offre_Link'])
    all_job_details = []

    for url in job_urls:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')

        job_details = {}
        
        # Ajouter le lien
        job_details["Offre_Link"] = url
        
        # Extraction des détails de l'offre d'emploi
        details_section = soup.find('ul', class_='text-small')
        if details_section:
            for li in details_section.find_all('li', class_='row-fluid'):
                key = li.find('span', class_='span4').text.strip()
                value = li.find('span', class_='span8').text.strip()
                job_details[key] = value

        # Extraction du texte fourni
        description_section = soup.find('div', class_='spaced details-description')
        if description_section:
            provided_text = description_section.text.strip()
            job_details['Provided Text'] = provided_text

        all_job_details.append(job_details)

    # Création d'un DataFrame pour les détails des offres d'emploi
    df_Novojob = pd.DataFrame(all_job_details)
    # Ajouter les listes existantes en tant que colonnes au DataFrame
    df_Novojob = pd.merge(df_offers,df_Novojob, on='Offre_Link')
    df_Novojob = df_Novojob.drop_duplicates()
    df_Novojob.reset_index(drop=True, inplace=True)

    
    equivalences = {
    "Offre_Link": "SITE_WEB_DE_L_ENTREPRISE",
    "Nom de l'entreprise": "RAISON_SOCIALE_DE_L_ENTREPRISE",
    "Secteur d'activité": "BRANCHE_D_ACTIVITE",
    "Lieu de travail": "LIEU_DU_POSTE_DE_TRAVAIL",
    "Date d'expiration": "DATE_D_EXPIRATION_DE_L_OFFRE",
    "Nombre de postes": "NOMBRE_DE_POSTES_A_POURVOIR",
    "Niveau de poste": None,
    "Niveau d'étude (diplôme)": "NIVEAU_D_ETUDES",
    "Type de contrat": "TYPE_DE_CONTRAT_DU_POSTE",
    "Provided Text": "Texte_fourni",
    "Intitule": "INTITULE_DU_POSTE",
    "Entreprise": None,
    "Pays": "PAYS_DU_POSTE_DE_TRAVAIL",
    "Date": "DATE_DE_DEBUT_DE_L_OFFRE",
    "Niveau": None,
    "Experience_lettre": None,
    "url_Lien": "Offre_Link1"}
    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme
    df_Novojob = renommer_colonnes(df_Novojob, equivalences)
    df_Novojob['NOMBRE_DE_POSTES_A_POURVOIR'] = df_Novojob['NOMBRE_DE_POSTES_A_POURVOIR'].str.extract(r"([0-9,]+)")
    df_Novojob[["VILLE_DU_POSTE_DE_TRAVAIL","PAYS_DU_POSTE_DE_TRAVAIL"]] = df_Novojob['LIEU_DU_POSTE_DE_TRAVAIL'].str.split(',', expand=True)
    for i in range(len(df_Novojob["PAYS_DU_POSTE_DE_TRAVAIL"])):
        if df_Novojob["PAYS_DU_POSTE_DE_TRAVAIL"][i] is None :
            df_Novojob.loc[i, "PAYS_DU_POSTE_DE_TRAVAIL"] = df_Novojob["VILLE_DU_POSTE_DE_TRAVAIL"][i]
            df_Novojob.loc[i, "VILLE_DU_POSTE_DE_TRAVAIL"]= " "


    # Réorganiser les colonnes selon vos besoins

    return df_Novojob

# Appel de la fonction principale
scrap_novojob=scrap_novojob()
scrap_novojob


,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,Niveau,Experience_lettre,url,SITE_WEB_DE_L_ENTREPRISE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,Niveau de poste,BRANCHE_D_ACTIVITE,Texte_fourni,Niveau d'étude (diplome),NOMBRE_DE_POSTES_A_POURVOIR,TYPE_DE_CONTRAT_DU_POSTE,RAISON_SOCIALE_DE_L_ENTREPRISE,VILLE_DU_POSTE_DE_TRAVAIL
0,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,Manager / Responsable département,Services,Entreprise spécialisée dans l’ingénierie mécan...,NaN,NaN,NaN,NaN,Abidjan
1,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,Manager / Responsable département,Services,Entreprise spécialisée dans l’ingénierie mécan...,NaN,NaN,NaN,NaN,Abidjan
2,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,Manager / Responsable département,Services,Entreprise spécialisée dans l’ingénierie mécan...,NaN,NaN,NaN,NaN,Abidjan
3,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,Manager / Responsable département,Services,Entreprise spécialisée dans l’ingénierie mécan...,NaN,NaN,NaN,NaN,Abidjan
4,Gestionnaire des Ressources Humaines,None,Côte d'ivoire,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Abidjan, Côte d'ivoire",22 Juin,Manager / Responsable département,Services,Entreprise spécialisée dans l’ingénierie mécan...,NaN,NaN,NaN,NaN,Abidjan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,Maintenance Officer,OLAM FOOD Ingrédients (OFI),Côte d'ivoire,31 Janvier,Confirmé / Expérimenté,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Bouake, Côte d'ivoire",30 Avril,Confirmé / Expérimenté,"Distribution, Commerce",Position within the structure :Position based ...,Formation Professionnelle,01,CDI,NaN,Bouake
767,Maintenance Officer,OLAM FOOD Ingrédients (OFI),Côte d'ivoire,31 Janvier,Confirmé / Expérimenté,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Bouake, Côte d'ivoire",30 Avril,Confirmé / Expérimenté,"Distribution, Commerce",Position within the structure :Position based ...,Formation Professionnelle,01,CDI,NaN,Bouake
768,Maintenance Officer,OLAM FOOD Ingrédients (OFI),Côte d'ivoire,31 Janvier,Confirmé / Expérimenté,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Bouake, Côte d'ivoire",30 Avril,Confirmé / Expérimenté,"Distribution, Commerce",Position within the structure :Position based ...,Formation Professionnelle,01,CDI,NaN,Bouake
769,Production in Charge,OLAM FOOD Ingrédients (OFI),Côte d'ivoire,06 Février,Confirmé / Expérimenté,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,"Bouake, Côte d'ivoire",06 Mai,Confirmé / Expérimenté,"Distribution, Commerce",Présentation de l’entreprise \nOLAM FOOD ingre...,"Master 1, Licence Bac + 4",01,CDI,NaN,Bouake


In [6]:
df=scrap_novojob

In [7]:
df[["VILLE_DU_POSTE_DE_TRAVAIL","PAYS_DU_POSTE_DE_TRAVAIL"]] = df['Niveau'].str.split('/', expand=True)
    

In [8]:
from plotly.io import write_html
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Charger les données (assurez-vous d'avoir préalablement importé les bibliothèques nécessaires et défini df_offers)
# df_offers = pd.read_csv("votre_fichier.csv")  # Remplacez "votre_fichier.csv" par votre fichier de données

# Réorganiser les colonnes selon vos besoins
scrap_novojob = scrap_novojob[[
    'INTITULE_DU_POSTE', 'Entreprise', 'PAYS_DU_POSTE_DE_TRAVAIL', 'DATE_DE_DEBUT_DE_L_OFFRE', 'Niveau', 'Experience_lettre',
    'url', "SITE_WEB_DE_L_ENTREPRISE", "RAISON_SOCIALE_DE_L_ENTREPRISE", "BRANCHE_D_ACTIVITE", "LIEU_DU_POSTE_DE_TRAVAIL",
    "DATE_D_EXPIRATION_DE_L_OFFRE", "NOMBRE_DE_POSTES_A_POURVOIR", "Niveau de poste", "Niveau d'étude (diplome)", "TYPE_DE_CONTRAT_DU_POSTE",
    "Texte_fourni", "VILLE_DU_POSTE_DE_TRAVAIL"
]]

# Définir une fonction pour attribuer des valeurs numériques à l'expérience
def experience_level(row):
    exp_mapping = {
        'Moins d’un an': 1, 'Sans expérience': 1, '1 à 2 ans': 1, '2 à 3 ans': 2, '3 à 4 ans': 3, '4 à 5 ans': 4,
        '3 à 5 ans': 4, '5 à 6 ans': 5, '6 à 7 ans': 6, '6 à 10 ans': 6, '7 à 8 ans': 7, '8 à 9 ans': 8,
        '9 à 10 ans': 9, 'Plus de 10 ans': 11
    }
    return exp_mapping.get(row['Experience_lettre'], 12)

# Appliquer la fonction pour créer une nouvelle colonne "Niveau_experience"
scrap_novojob['Niveau_experience'] = scrap_novojob.apply(experience_level, axis=1)

# Options pour les filtres
secteurs_activite_options = [{'label': 'Tout sélectionner', 'value': 'all'}] + [{'label': secteur, 'value': secteur} for secteur in scrap_novojob["BRANCHE_D_ACTIVITE"].unique()]
types_contrat_options = [{'label': 'Tout sélectionner', 'value': 'all'}] + [{'label': contrat, 'value': contrat} for contrat in scrap_novojob["TYPE_DE_CONTRAT_DU_POSTE"].unique() if pd.notna(contrat)]
niveaux_experience_options = [{'label': level, 'value': level} for level in sorted(set(scrap_novojob['Niveau']))]

# Palettes de couleurs
secteur_activite_palette = px.colors.qualitative.Pastel1
entreprise_palette = px.colors.qualitative.Set2
type_contrat_palette = px.colors.qualitative.Pastel2

# Initialiser l'application Dash avec les styles CSS
external_stylesheets = ['https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# Définir la mise en page du tableau de bord
app.layout = html.Div(children=[
    html.H1(children='Tableau de Bord des Offres d\'Emploi', className='text-center mb-4', style={'color': 'green'}),

    # Filtres interactifs
    html.Div([
        html.Div([
            html.Label('Filtrer par Secteur d\'activité', style={'color': 'white'}),
            dcc.Dropdown(
                id='filtre-secteur-activite',
                options=secteurs_activite_options,
                value='Tous les secteurs d\'activité',
                className='form-control'
            ),
        ], className='col-md-4'),

        html.Div([
            html.Label('Filtrer par Type de contrat', style={'color': 'white'}),
            dcc.Dropdown(
                id='filtre-type-contrat',
                options=types_contrat_options,
                value='Tous les types de contrat',
                className='form-control'
            ),
        ], className='col-md-4'),

        html.Div([
            html.Label('Filtrer par Niveau d\'expérience', style={'color': 'white'}),
            dcc.Dropdown(
                id='filtre-niveau-experience',
                options=niveaux_experience_options,
                value='Tous les niveaux d\'expérience',
                className='form-control'
            ),
        ], className='col-md-4'),

        html.Button('Réinitialiser les filtres', id='bouton-reset', n_clicks=0, style={'background-color': 'green'})
    ], style={'display': 'flex', 'background-color': 'green'}),

    # Graphiques
    html.Div([
        dcc.Graph(id='graph-secteur-activite', className='col-md-6'),
        dcc.Graph(id='graph-entreprise', className='col-md-6'),
        dcc.Graph(id='graph-type-contrat', className='col-md-6'),
        dcc.Graph(id='graph-part-entreprises-par-type', className='col-md-6'),
        dcc.Graph(id='graph-taille-moyenne-entreprises', className='col-md-6'),
        dcc.Graph(id='graph-part-entreprises-par-categorie', className='col-md-6'),
        
    ],className='row', style={'background-color': 'white'})
])

# Callback pour mettre à jour les graphiques en fonction des filtres
@app.callback(
    [Output('graph-secteur-activite', 'figure'),
     Output('graph-entreprise', 'figure'),
     Output('graph-type-contrat', 'figure'),
     Output('graph-part-entreprises-par-type', 'figure'),
     Output('graph-part-entreprises-par-categorie', 'figure'),
     Output('graph-taille-moyenne-entreprises', 'figure')],
    [Input('filtre-secteur-activite', 'value'),
     Input('filtre-type-contrat', 'value'),
     Input('filtre-niveau-experience', 'value'),
     Input('bouton-reset', 'n_clicks')]
)
def update_graphs(secteurs_activite, types_contrat, niveaux_experience, n_clicks_reset):
    # Filtrage du DataFrame en fonction des sélections
    df_filtered = scrap_novojob
    # Définir les filtres à appliquer
    filters = {
        'BRANCHE_D_ACTIVITE': secteurs_activite,
        'TYPE_DE_CONTRAT_DU_POSTE': types_contrat,
        'Niveau': niveaux_experience
    }

    # Appliquer les filtres
    for column, values in filters.items():
        if values and 'Tous' not in values:
            if isinstance(values, str):
                values = [values]  # Transformer en liste si c'est une chaîne de caractères unique
            df_filtered = df_filtered[df_filtered[column].isin(values)]

    # Graphique du nombre d'offres par secteur d'activité
    fig_secteur_activite = px.bar(df_filtered, x="BRANCHE_D_ACTIVITE", title="Nombre d'offres par secteur d'activité",
                                    color_discrete_sequence=['orange'])

    # Graphique du nombre d'offres par entreprise
    fig_entreprise = px.bar(df_filtered, x="Entreprise", title="Nombre d'offres par entreprise",
                            color_discrete_sequence=['orange'])

    # Graphique du type de contrat proposé
    fig_type_contrat = px.bar(df_filtered, x="TYPE_DE_CONTRAT_DU_POSTE", title="Type de contrat proposé",
                               color_discrete_sequence=['orange'])

    # Graphique pour la part des entreprises par type de contrat
    fig_part_entreprises_par_type = px.pie(df_filtered, names="TYPE_DE_CONTRAT_DU_POSTE",
                                           title="Part des entreprises par type de contrat", color_discrete_sequence=['orange'])

    # Graphique pour la part des entreprises par catégorie
    fig_part_entreprises_par_categorie = px.pie(df_filtered, names="Niveau de poste",
                                                title="Part des entreprises par catégorie", color_discrete_sequence=['orange'])

    # Graphique pour la taille moyenne des entreprises par catégorie
    fig_taille_moyenne_entreprises = px.bar(df_filtered, x="Niveau de poste", y="NOMBRE_DE_POSTES_A_POURVOIR",
                                             title="Taille moyenne des entreprises par catégorie", color_discrete_sequence=['orange'])

    return fig_secteur_activite, fig_entreprise, fig_type_contrat, fig_part_entreprises_par_type, fig_part_entreprises_par_categorie, fig_taille_moyenne_entreprises

if __name__ == '__main__':
    app.run_server(debug=True, port=8058)

    # Récupérer les figures après l'exécution de l'application
    fig_secteur_activite, fig_entreprise, fig_type_contrat, fig_part_entreprises_par_type, fig_part_entreprises_par_categorie, fig_taille_moyenne_entreprises = update_graphs(
        None, None, None, None)

    # Sauvegarder les graphiques au format HTML
    write_html(fig_secteur_activite, 'graph_secteur_activite.html')
    write_html(fig_entreprise, 'graph_entreprise.html')
    write_html(fig_type_contrat, 'graph_type_contrat.html')
    write_html(fig_part_entreprises_par_type, 'graph_part_entreprises_par_type.html')
    write_html(fig_part_entreprises_par_categorie, 'graph_part_entreprises_par_categorie.html')
    write_html(fig_taille_moyenne_entreprises, 'graph_taille_moyenne_entreprises.html')


C:\Users\Dell\AppData\Local\Temp\ipykernel_6808\2193586182.py:3: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

C:\Users\Dell\AppData\Local\Temp\ipykernel_6808\2193586182.py:4: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [9]:
import pandas as pd

# Afficher les statistiques descriptives
descriptive_stats = scrap_novojob.describe(include='all')
descriptive_stats


,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,Niveau,Experience_lettre,url,SITE_WEB_DE_L_ENTREPRISE,RAISON_SOCIALE_DE_L_ENTREPRISE,BRANCHE_D_ACTIVITE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,NOMBRE_DE_POSTES_A_POURVOIR,Niveau de poste,Niveau d'étude (diplome),TYPE_DE_CONTRAT_DU_POSTE,Texte_fourni,VILLE_DU_POSTE_DE_TRAVAIL,Niveau_experience
count,786,773,558,786,786,786,786,786,118,786,786,786,773,786,611,647,786,786,786.000000
unique,113,44,4,48,7,6,27,120,2,8,7,49,10,19,21,10,120,7,NaN
top,Commercial H/F,Exceliam,Expérimenté,07 Février,Confirmé / Expérimenté,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Société Générale Côte D'ivoire,Services,"Abidjan, Côte d'ivoire",05 Mai,01,Confirmé / Expérimenté,"Master 1, Licence Bac + 4| Master 2, Ingénior...",CDI,La banque en toute confiance.Acteur majeur du ...,Confirmé,NaN
freq,18,156,356,73,356,295,50,16,81,302,434,56,639,293,183,319,16,356,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.333333
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.287230
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000


In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Exemple de modélisation simple (n'utilisez que les variables numériques)
X = scrap_novojob[['NOMBRE_DE_POSTES_A_POURVOIR']]
y = scrap_novojob['NOMBRE_DE_POSTES_A_POURVOIR']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Création et ajustement du modèle de régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
predictions = model.predict(X_test)

# Évaluation du modèle
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [100]:
import spacy

# Charger le modèle NLP
nlp = spacy.load("fr_core_news_sm")

# Appliquer NLP sur le texte de l'offre
texte_offre = scrap_novojob['Texte_fourni'].iloc[0]  # Utilisez l'indice approprié
doc = nlp(texte_offre)

# Extraire les entités nommées
entites = [(ent.text, ent.label_) for ent in doc.ents]
print("Entités nommées :", entites)

# Extraire les compétences
competences = [token.text for token in doc if "compétence" in token.text.lower()]
print("Compétences requises :", competences)

# D'autres analyses NLP peuvent également être appliquées selon les besoins.


Entités nommées : [('H/F)Vous êtes', 'ORG'), ('soyez', 'MISC'), ('LM', 'MISC'), ('RémunérationSalaire', 'MISC'), ('Lundi', 'PER'), ('Koumassi', 'LOC')]
Compétences requises : []


In [19]:
#scrap_novojob["NOMBRE_DE_POSTES_A_POURVOIR"][0] affiche ça 40 postes ouverts mais je souhaite qui affiche que 40

# Emploi.educarriere.ci

In [ ]:
#emploi_educarriere affiche BAC+5, BAC+4, BAC+3, BAC+2, BAC+1, BAC, BT

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
import urllib3

def emploi_educarriere():
    # Ignorer les avertissements SSL
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
    # Fonction pour extraire le texte d'un élément HTML
    def extract_text(element, class_name=None, style=None, text_contains=None):
        if element:
            tag = element.find(class_=class_name, style=style, text=text_contains)
            return tag.text.strip() if tag else ""
        else:
            return ""

    # Fonction pour nettoyer le texte
    def clean_text(text):
        if text is not None:
            cleaned_text = text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')
            return cleaned_text.strip() if cleaned_text else None
        else:
            return None

    # Fonction pour extraire la date à partir d'un élément HTML
    def extract_date(element, text_contains):
        date_elements = element.find_all('a', class_='text')
        date = next((e.find('span', style='color:#FF0000;').text.strip() for e in date_elements if text_contains in e.text), "")
        return date

    # Fonction pour extraire les détails des offres d'emploi
    def scrap_job_details(urls):
        # En-tête pour éviter d'être bloqué
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        options = webdriver.ChromeOptions()
        options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
        options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
        chrome_driver_path = 'C:\\Program Files (x86)\\chromedriver.exe'
        options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
        options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
        driver = webdriver.Chrome(options=options)

        # Liste pour stocker les détails de chaque emploi
        all_job_details = []

        # Parcourir les liens
        for url in urls:
            req = requests.get(url, headers=headers)
            soup = BeautifulSoup(req.text, 'html.parser')
            time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

            offres = soup.find_all('div', class_='box row')

            # Parcourir les offres d'emploi sur la page principale
            for offre in offres:
                # Trouver la balise <h4> dans la structure HTML pour extraire le lien
                offre_link_tag = offre.find('h4')

                # Vérifier si la balise <h4> a été trouvée
                if offre_link_tag:
                    # Extraire le lien de l'attribut 'href'
                    offre_link = offre_link_tag.find('a')['href']
                    all_job_details.append({'Offre_Link': offre_link})

        # Fermer le pilote Selenium à la fin
        driver.quit()

        # Concaténer tous les détails des emplois en un seul DataFrame
        if all_job_details:
            all_job_details_df = pd.DataFrame(all_job_details)
            return all_job_details_df
        else:
            print("Aucun détail d'offre d'emploi trouvé.")
            return None

    # Fonction pour récupérer les données des offres d'emploi
    def scrape_emploi_ci(url):
        try:
            response = requests.get(url, timeout=500)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion à {url} : {e}")
            return pd.DataFrame()

        soup = BeautifulSoup(response.text, 'html.parser')

        job_description_wrappers = soup.find_all('div', class_='box row')

        data_list = []

        for wrapper in job_description_wrappers:
            h4_tag = wrapper.find('h4')
            poste = extract_text(h4_tag)

            entry_title_tag = wrapper.find('p', class_='entry-title')
            sous_titre = extract_text(entry_title_tag)

            a_text_tag = wrapper.find('a', class_='text')
            code = extract_text(a_text_tag, style='color:#FF0000;')

            date_edition = extract_date(wrapper, "Date d'édition:")
            date_limite = extract_date(wrapper, "Date limite:")

            pays_tag = wrapper.find('a', class_='text')
            pays = pays_tag.find_parent().text.strip().split()[-1] if pays_tag else None

            sous_titre = clean_text(sous_titre)

            data_list.append({
                'Poste': clean_text(poste),
                'Sous_titre': sous_titre,
                'Code': clean_text(code),
                'Date_DEdition': date_edition,
                'Date_limite': date_limite,
                'Pays': clean_text(pays),
                'URL': url  # Ajout de la colonne 'URL'
            })

        df = pd.DataFrame(data_list)
        return df

    # Fonction pour ajouter la colonne 'Offre_Link' à un DataFrame existant
    def add_offre_link_column(result_df):
        job_details_df = scrap_job_details(list(result_df["URL"]))
        if job_details_df is not None:
            # Join des DataFrames
            result_df = pd.concat([result_df, job_details_df], axis=1)
            return result_df
        else:
            print("Impossible d'ajouter la colonne 'Offre_Link' au DataFrame.")
            return None

    # Fonction pour extraire les informations de l'offre d'emploi
    def extract_job_information(soup, url):
        try:
            # Extraction des informations de l'offre d'emploi
            poste = soup.select_one('li.list-group-item:-soup-contains("Poste")').strong.next_sibling.strip()
            type_offre = soup.select_one('li.list-group-item:-soup-contains("Type d\'offre")').strong.next_sibling.strip()
            metiers = soup.select_one('li.list-group-item:-soup-contains("Métier(s):")').strong.next_sibling.strip()
            niveaux = soup.select_one('li.list-group-item:-soup-contains("Niveau(x):")').strong.next_sibling.strip()
            experience = soup.select_one('li.list-group-item:-soup-contains("Expérience:")').strong.next_sibling.strip()
            lieu = soup.select_one('li.list-group-item:-soup-contains("Lieu:")').strong.next_sibling.strip()
            
            # Extraction des dates de publication et de limite
            date_publication = soup.find('strong', string='Date de publication:').find_next('span').text.strip()
            date_limite = soup.find('strong', string='Date limite:').find_next('span').text.strip()
            
            # description = soup.select_one('div.text-col.post.small-post.col-md-9.col-xs-12 ul.list-group').text.strip()
            description = soup.select_one('div.entry-content').text.strip()

            return {
                "Poste": [poste],
                "Type d'offre": [type_offre],
                "Métier(s)": [metiers],
                "Niveau(x)": [niveaux],
                "Expérience": [experience],
                "Lieu": [lieu],
                "Offre_Link": [url],
                "Date de publication": [date_publication],
                "Date limite": [date_limite],
                "Description": [description]
            }
        except Exception as e:
            print(f"An error occurred while extracting job information for URL {url}: {e}")
            # Retourner un dictionnaire avec l'URL en cas d'erreur
            return {"Offre_Link": [url]}

    # Liste des liens
    urls = ["https://emploi.educarriere.ci/nos-offres?page1={}&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur=".format(category) for category in range(41)]

    # Créer un DataFrame à partir des liens
    result_df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)

    # Ajouter la colonne 'Offre_Link'
    result_df = add_offre_link_column(result_df)
    # Supprimer les lignes avec plus de 80% de valeurs NaN
    threshold = int(result_df.shape[1] * 0.8)  # 80% des colonnes
    result_df = result_df.dropna(thresh=threshold)
    result_df = result_df.reset_index(drop=True)
    
    # Liste des URLs à scraper
    urls = list(result_df['Offre_Link'])

    # Liste pour stocker les DataFrames
    dfs = []

    # Boucle sur chaque URL
    for url in urls:
        try:
            # Envoyer une requête GET au site avec un délai de 120 secondes
            response = requests.get(url, headers=headers, verify=True, timeout=120)

            # Vérifier si la requête a réussi (statut 200)
            if response.status_code == 200:
                # Analyser le contenu de la page avec BeautifulSoup
                soup = BeautifulSoup(response.text, 'html.parser')

                # Extraire les informations sur l'emploi
                job_info = extract_job_information(soup, url)

                # Créer un DataFrame
                df = pd.DataFrame(job_info)

                # Ajouter le DataFrame à la liste
                dfs.append(df)
            else:
                print(f"Échec de la requête pour l'URL {url}. Statut : {response.status_code}")
                # Ajouter une ligne avec l'URL en cas d'erreur
                dfs.append(pd.DataFrame({"Offre_Link": [url]}))

        except requests.exceptions.Timeout:
            print(f"Timeout lors de la requête pour l'URL {url}")
            # Ajouter une ligne avec l'URL en cas d'erreur
            dfs.append(pd.DataFrame({"Offre_Link": [url]}))
        except requests.exceptions.RequestException as e:
            print(f"Une erreur s'est produite lors de la requête pour l'URL {url}: {e}")
            # Ajouter une ligne avec l'URL en cas d'erreur
            dfs.append(pd.DataFrame({"Offre_Link": [url]}))

        # Ajouter un délai de 5 secondes entre les requêtes pour éviter d'être bloqué
        time.sleep(5)

    # Concaténer tous les DataFrames en un seul DataFrame
    df_Educarriere = pd.concat(dfs, ignore_index=True)

    # Ajouter les listes existantes en tant que colonnes au DataFrame
    df_Educarriere = pd.merge(df_Educarriere, result_df, on='Offre_Link')
    df_Educarriere = df_Educarriere.drop_duplicates()
    df_Educarriere.reset_index(drop=True, inplace=True)
    
    equivalences = {
    "Poste_y": "INTITULE_DU_POSTE",
    "Type d'offre": "TYPE_DE_CONTRAT_DU_POSTE",
    "Métier(s)": "SPECIALITE",
    "Niveau(x)": "DIPLOME",
    "Expérience": "EXPERIENCE_PROFESSIONNELLE",
    "Lieu": "LIEU_DU_POSTE_DE_TRAVAIL",
    "Offre_Link": "SITE_WEB_DE_L_ENTREPRISE",
    "Date de publication": "DATE_DE_DEBUT_DE_L_OFFRE",
    "Date limite": "DATE_D_EXPIRATION_DE_L_OFFRE",
    "Poste_x": "BRANCHE_D_ACTIVITE",
    "Description": None,
    "Sous_titre": None,
    "Code": None,
    "Date_DEdition": None,
    "Date_limite": None,
    "Pays": "PAYS_DU_POSTE_DE_TRAVAIL"}
    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme
    
    
    # Renommer les colonnes du DataFrame
    df_Educarriere = renommer_colonnes(df_Educarriere, equivalences)
    df_Educarriere[["EXPERIENCE_PROFESSIONNELLE", 'Unite_EXPERIENCE_PROFESSIONNELLE']] = df_Educarriere["EXPERIENCE_PROFESSIONNELLE"].str.extract(r"([0-9]+)\s*([a-zA-Z]+)")


    # Réorganiser les colonnes selon vos besoins
    # Vous pouvez réorganiser les colonnes ici

    # Afficher ou retourner le DataFrame selon votre besoin
    return df_Educarriere

# Appel de la fonction principale

df_emploi_educarriere=emploi_educarriere()
df_emploi_educarriere

Erreur de connexion à https://emploi.educarriere.ci/nos-offres?page1=30&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur= : HTTPSConnectionPool(host='emploi.educarriere.ci', port=443): Read timed out. (read timeout=500)
Une erreur s'est produite lors de la requête pour l'URL https://emploi.educarriere.ci/offre-115972-chef-cuisinier-a-domicile.html: ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))
An error occurred while extracting job information for URL https://emploi.educarriere.ci/offre-115981-chef-machinistre.html: 'NoneType' object has no attribute 'text'


,BRANCHE_D_ACTIVITE,TYPE_DE_CONTRAT_DU_POSTE,SPECIALITE,DIPLOME,EXPERIENCE_PROFESSIONNELLE,LIEU_DU_POSTE_DE_TRAVAIL,SITE_WEB_DE_L_ENTREPRISE,DATE_DE_DEBUT_DE_L_OFFRE,DATE_D_EXPIRATION_DE_L_OFFRE,Description,INTITULE_DU_POSTE,Sous_titre,Code,Date_DEdition,Date_limite,PAYS_DU_POSTE_DE_TRAVAIL,URL,Unite_EXPERIENCE_PROFESSIONNELLE
0,DEVELOPPEUR DAPPLICATION FULL STACK,Emploi,Informatique,BAC+4,5,Côte d'Ivoire,https://emploi.educarriere.ci/offre-115980-dev...,04/04/2024,30/04/2024,PCPLUSGROUPHOLDINGrecruteDEVELOPPEUR D’APPLICA...,CHEF MACHINISTRE,Description du posteNous recherchons un(e) mac...,111804,04/04/2024,15/04/2024,abidjan,https://emploi.educarriere.ci/nos-offres?page1...,ans
1,DEVELOPPEUR DAPPLICATION FULL STACK,Emploi,Informatique,BAC+4,5,Côte d'Ivoire,https://emploi.educarriere.ci/offre-115980-dev...,04/04/2024,30/04/2024,PCPLUSGROUPHOLDINGrecruteDEVELOPPEUR D’APPLICA...,RESPONSABLE DEVELOPPEMENT COMMERCIAL,AGENCE IVOIRE INTERIM (A2I)recruteRESPONSABLE ...,111771,03/04/2024,30/04/2024,ABIDJAN,https://emploi.educarriere.ci/nos-offres?page1...,ans
2,DEVELOPPEUR DAPPLICATION FULL STACK,Emploi,Informatique,BAC+4,5,Côte d'Ivoire,https://emploi.educarriere.ci/offre-115980-dev...,04/04/2024,30/04/2024,PCPLUSGROUPHOLDINGrecruteDEVELOPPEUR D’APPLICA...,STAGIAIRE COMPTABLE,CGArecruteSTAGIAIRE COMPTABLEDescription du po...,111757,03/04/2024,28/04/2024,cam...,https://emploi.educarriere.ci/nos-offres?page1...,ans
3,DEVELOPPEUR DAPPLICATION FULL STACK,Emploi,Informatique,BAC+4,5,Côte d'Ivoire,https://emploi.educarriere.ci/offre-115980-dev...,04/04/2024,30/04/2024,PCPLUSGROUPHOLDINGrecruteDEVELOPPEUR D’APPLICA...,COURSIER A MOTO,NISSIM ConsultingrecruteCOURSIER A MOTODescrip...,111739,03/04/2024,15/04/2024,ABIDJAN,https://emploi.educarriere.ci/nos-offres?page1...,ans
4,DEVELOPPEUR DAPPLICATION FULL STACK,Emploi,Informatique,BAC+4,5,Côte d'Ivoire,https://emploi.educarriere.ci/offre-115980-dev...,04/04/2024,30/04/2024,PCPLUSGROUPHOLDINGrecruteDEVELOPPEUR D’APPLICA...,CHAUFFEUR-COURSIER,Tanasa Real EstaterecruteCHAUFFEUR-COURSIERDes...,111724,03/04/2024,15/04/2024,Abidjan,https://emploi.educarriere.ci/nos-offres?page1...,ans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,Regional Recruiter  West Africa Region,Emploi,"Ressources Humaines , Sciences sociales","BAC+3, BAC+4, BAC+5",5,West Africa,https://emploi.educarriere.ci/offre-115939-reg...,03/04/2024,07/04/2024,CATHOLIC RELIEF SERVICES/COTE D'IVOIRErecruitR...,FORMATEUR EN ETUDE D'IMPACT ENVIRONNEMENTAL ET...,INSTITUT PANAFRICAIN DES HAUTES ETUDES ET FORM...,107754,18/12/2023,01/08/2024,Abidjan,https://emploi.educarriere.ci/nos-offres?page1...,ans
547,Regional Recruiter  West Africa Region,Emploi,"Ressources Humaines , Sciences sociales","BAC+3, BAC+4, BAC+5",5,West Africa,https://emploi.educarriere.ci/offre-115939-reg...,03/04/2024,07/04/2024,CATHOLIC RELIEF SERVICES/COTE D'IVOIRErecruitR...,ASSISTANT(E),Beryl InformatiquerecruteASSISTANT(E)Descripti...,102798,04/08/2023,14/08/2024,Blockhaus...,https://emploi.educarriere.ci/nos-offres?page1...,ans
548,Secrétaire de Direction,Emploi,Secrétariat,BAC+2,3,bonoua yaou 8 KILO Carrefour réma,https://emploi.educarriere.ci/offre-115938-rec...,03/04/2024,15/04/2024,UNI TECHNOLOGYrecruteSecrétaire de Direction\n...,PLUSIEURS PROFILS,"Description du posteDepuis plus de 20 ans, l h...",108445,09/01/2024,30/06/2024,Indu...,https://emploi.educarriere.ci/nos-offres?page1...,ans
549,Secrétaire de Direction,Emploi,Secrétariat,BAC+2,3,bonoua yaou 8 KILO Carrefour réma,https://emploi.educarriere.ci/offre-115938-rec...,03/04/2024,15/04/2024,UNI TECHNOLOGYrecruteSecrétaire de Direction\n...,FORMATEUR EN COMMUNICATION ET GESTION DE CRISE,INSTITUT PANAFRICAIN DES HAUTES ETUDES ET FORM...,107753,18/12/2023,01/08/2024,Abidjan,https://emploi.educarriere.ci/nos-offres?page1...,ans


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
import urllib3

def emploi_educarriere():
    # Ignorer les avertissements SSL
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
    # Fonction pour extraire le texte d'un élément HTML
    def extract_text(element, class_name=None, style=None, text_contains=None):
        if element:
            tag = element.find(class_=class_name, style=style, text=text_contains)
            return tag.text.strip() if tag else ""
        else:
            return ""

    # Fonction pour nettoyer le texte
    def clean_text(text):
        if text is not None:
            cleaned_text = text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')
            return cleaned_text.strip() if cleaned_text else None
        else:
            return None

    # Fonction pour extraire la date à partir d'un élément HTML
    def extract_date(element, text_contains):
        date_elements = element.find_all('a', class_='text')
        date = next((e.find('span', style='color:#FF0000;').text.strip() for e in date_elements if text_contains in e.text), "")
        return date

    # Fonction pour extraire les détails des offres d'emploi
    def scrap_job_details(urls):
        # En-tête pour éviter d'être bloqué
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        options = webdriver.ChromeOptions()
        options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
        options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
        chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
        options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
        options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
        driver = webdriver.Chrome(options=options)

        # Liste pour stocker les détails de chaque emploi
        all_job_details = []

        # Parcourir les liens
        for url in urls:
            req = requests.get(url, headers=headers)
            soup = BeautifulSoup(req.text, 'html.parser')
            time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

            offres = soup.find_all('div', class_='box row')

            # Parcourir les offres d'emploi sur la page principale
            for offre in offres:
                # Trouver la balise <h4> dans la structure HTML pour extraire le lien
                offre_link_tag = offre.find('h4')

                # Vérifier si la balise <h4> a été trouvée
                if offre_link_tag:
                    # Extraire le lien de l'attribut 'href'
                    offre_link = offre_link_tag.find('a')['href']
                    all_job_details.append({'Offre_Link': offre_link})

        # Fermer le pilote Selenium à la fin
        driver.quit()

        # Concaténer tous les détails des emplois en un seul DataFrame
        if all_job_details:
            all_job_details_df = pd.DataFrame(all_job_details)
            return all_job_details_df
        else:
            print("Aucun détail d'offre d'emploi trouvé.")
            return None

    # Fonction pour récupérer les données des offres d'emploi
    def scrape_emploi_ci(url):
        try:
            response = requests.get(url, timeout=200)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion à {url} : {e}")
            return pd.DataFrame()

        soup = BeautifulSoup(response.text, 'html.parser')

        job_description_wrappers = soup.find_all('div', class_='box row')

        data_list = []

        for wrapper in job_description_wrappers:
            h4_tag = wrapper.find('h4')
            poste = extract_text(h4_tag)

            entry_title_tag = wrapper.find('p', class_='entry-title')
            sous_titre = extract_text(entry_title_tag)

            a_text_tag = wrapper.find('a', class_='text')
            code = extract_text(a_text_tag, style='color:#FF0000;')

            date_edition = extract_date(wrapper, "Date d'édition:")
            date_limite = extract_date(wrapper, "Date limite:")

            pays_tag = wrapper.find('a', class_='text')
            pays = pays_tag.find_parent().text.strip().split()[-1] if pays_tag else None

            sous_titre = clean_text(sous_titre)

            data_list.append({
                'Poste': clean_text(poste),
                'Sous_titre': sous_titre,
                'Code': clean_text(code),
                'Date_DEdition': date_edition,
                'Date_limite': date_limite,
                'Pays': clean_text(pays),
                'URL': url  # Ajout de la colonne 'URL'
            })

        df = pd.DataFrame(data_list)
        return df

    # Fonction pour ajouter la colonne 'Offre_Link' à un DataFrame existant
    def add_offre_link_column(result_df):
        job_details_df = scrap_job_details(list(result_df["URL"]))
        if job_details_df is not None:
            # Join des DataFrames
            result_df = pd.concat([result_df, job_details_df], axis=1)
            return result_df
        else:
            print("Impossible d'ajouter la colonne 'Offre_Link' au DataFrame.")
            return None

    # Fonction pour extraire les informations de l'offre d'emploi
    def extract_job_information(soup, url):
        try:
            # Extraction des informations de l'offre d'emploi
            poste = soup.select_one('li.list-group-item:-soup-contains("Poste")').strong.next_sibling.strip()
            type_offre = soup.select_one('li.list-group-item:-soup-contains("Type d\'offre")').strong.next_sibling.strip()
            metiers = soup.select_one('li.list-group-item:-soup-contains("Métier(s):")').strong.next_sibling.strip()
            niveaux = soup.select_one('li.list-group-item:-soup-contains("Niveau(x):")').strong.next_sibling.strip()
            experience = soup.select_one('li.list-group-item:-soup-contains("Expérience:")').strong.next_sibling.strip()
            lieu = soup.select_one('li.list-group-item:-soup-contains("Lieu:")').strong.next_sibling.strip()
            
            # Extraction des dates de publication et de limite
            date_publication = soup.find('strong', string='Date de publication:').find_next('span').text.strip()
            date_limite = soup.find('strong', string='Date limite:').find_next('span').text.strip()
            
            # description = soup.select_one('div.text-col.post.small-post.col-md-9.col-xs-12 ul.list-group').text.strip()
            description = soup.select_one('div.entry-content').text.strip()

            return {
                "Poste": [poste],
                "Type d'offre": [type_offre],
                "Métier(s)": [metiers],
                "Niveau(x)": [niveaux],
                "Expérience": [experience],
                "Lieu": [lieu],
                "Offre_Link": [url],
                "Date de publication": [date_publication],
                "Date limite": [date_limite],
                "Description": [description]
            }
        except Exception as e:
            print(f"An error occurred while extracting job information for URL {url}: {e}")
            # Retourner un dictionnaire avec l'URL en cas d'erreur
            return {"Offre_Link": [url]}

    # Liste des liens
    urls = ["https://emploi.educarriere.ci/nos-offres?page1={}&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur=".format(category) for category in range(40)]

    # Créer un DataFrame à partir des liens
    result_df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)

    # Ajouter la colonne 'Offre_Link'
    result_df = add_offre_link_column(result_df)
    # Supprimer les lignes avec plus de 80% de valeurs NaN
    threshold = int(result_df.shape[1] * 0.8)  # 80% des colonnes
    result_df = result_df.dropna(thresh=threshold)
    result_df = result_df.reset_index(drop=True)
    
    # Liste des URLs à scraper
    urls = list(result_df['Offre_Link'])

    # Liste pour stocker les DataFrames
    dfs = []

    # Boucle sur chaque URL
    for url in urls:
        try:
            # Envoyer une requête GET au site avec un délai de 120 secondes
            response = requests.get(url, headers=headers, verify=True, timeout=120)

            # Vérifier si la requête a réussi (statut 200)
            if response.status_code == 200:
                # Analyser le contenu de la page avec BeautifulSoup
                soup = BeautifulSoup(response.text, 'html.parser')

                # Extraire les informations sur l'emploi
                job_info = extract_job_information(soup, url)

                # Créer un DataFrame
                df = pd.DataFrame(job_info)

                # Ajouter le DataFrame à la liste
                dfs.append(df)
            else:
                print(f"Échec de la requête pour l'URL {url}. Statut : {response.status_code}")
                # Ajouter une ligne avec l'URL en cas d'erreur
                dfs.append(pd.DataFrame({"Offre_Link": [url]}))

        except requests.exceptions.Timeout:
            print(f"Timeout lors de la requête pour l'URL {url}")
            # Ajouter une ligne avec l'URL en cas d'erreur
            dfs.append(pd.DataFrame({"Offre_Link": [url]}))
        except requests.exceptions.RequestException as e:
            print(f"Une erreur s'est produite lors de la requête pour l'URL {url}: {e}")
            # Ajouter une ligne avec l'URL en cas d'erreur
            dfs.append(pd.DataFrame({"Offre_Link": [url]}))

        # Ajouter un délai de 5 secondes entre les requêtes pour éviter d'être bloqué
        time.sleep(5)

    # Concaténer tous les DataFrames en un seul DataFrame
    df_Educarriere = pd.concat(dfs, ignore_index=True)

    # Ajouter les listes existantes en tant que colonnes au DataFrame
    df_Educarriere['Poste'] = list(result_df['Poste'])
    df_Educarriere['Sous_titre'] = list(result_df['Sous_titre'])
    df_Educarriere['Code'] = list(result_df['Code'])
    df_Educarriere['Date_DEdition'] = list(result_df['Date_DEdition'])
    df_Educarriere['Date_limite'] = list(result_df['Date_limite'])
    df_Educarriere['Pays'] = list(result_df['Pays'])
    equivalences = {
    "Poste": "INTITULE_DU_POSTE",
    "Type d'offre": "TYPE_DE_CONTRAT_DU_POSTE",
    "Métier(s)": "SPECIALITE",
    "Niveau(x)": None,
    "Expérience": "EXPERIENCE_PROFESSIONNELLE",
    "Lieu": "LIEU_DU_POSTE_DE_TRAVAIL",
    "Offre_Link": "SITE_WEB_DE_L_ENTREPRISE",
    "Date de publication": "DATE_DE_DEBUT_DE_L_OFFRE",
    "Date limite": "DATE_D_EXPIRATION_DE_L_OFFRE",
    "Description": None,
    "Sous_titre": None,
    "Code": None,
    "Date_DEdition": None,
    "Date_limite": None,
    "Pays": "PAYS_DU_POSTE_DE_TRAVAIL"}
    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme
    
    
    # Renommer les colonnes du DataFrame
    df_Educarriere = renommer_colonnes(df_Educarriere, equivalences)
    df_Educarriere[["EXPERIENCE_PROFESSIONNELLE", 'Unite_EXPERIENCE_PROFESSIONNELLE']] = df_Educarriere["EXPERIENCE_PROFESSIONNELLE"].str.extract(r"([0-9]+)\s*([a-zA-Z]+)")


    # Réorganiser les colonnes selon vos besoins
    # Vous pouvez réorganiser les colonnes ici

    # Afficher ou retourner le DataFrame selon votre besoin
    return df_Educarriere

# Appel de la fonction principale

emploi_educarriere=emploi_educarriere()
emploi_educarriere

In [33]:
# Importer les bibliothèques nécessaires
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Charger les données
df_offres = emploi_educarriere.head()

# Initialiser l'application Dash avec les styles CSS
external_stylesheets = ['https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# Définir la mise en page du tableau de bord
app.layout = html.Div(children=[
    html.H1(children='Tableau de Bord des Offres d\'Emploi', className='text-center mb-4', style={'color': 'green'}),

    # Filtres interactifs
    html.Div([
        html.Div([
            html.Label('Filtrer par Secteur d\'activité:', className='font-weight-bold', style={'color': 'white'}),
            dcc.Dropdown(
                id='dropdown-filtre-secteur',
                options=[{'label': secteur, 'value': secteur} for secteur in df_offres['BRANCHE_D_ACTIVITE'].unique()],
                value='Tous les secteurs',
                className='form-control'
            ),
        ], className='col-md-4'),

        html.Div([
            html.Label('Filtrer par Type de contrat:', className='font-weight-bold', style={'color': 'white'}),
            dcc.Dropdown(
                id='dropdown-filtre-type-contrat',
                options=[{'label': contrat, 'value': contrat} for contrat in df_offres['TYPE_DE_CONTRAT_DU_POSTE'].unique()],
                value='Tous les types de contrat',
                className='form-control'
            ),
        ], className='col-md-4'),
        
        html.Div([
            html.Label('Filtrer par DIPLOME:', className='font-weight-bold', style={'color': 'white'}),
            dcc.Dropdown(
                id='dropdown-filtre-diplome',
                options=[{'label': diplome, 'value': diplome} for diplome in df_offres['DIPLOME'].unique()],
                value='Tous les diplômes',
                className='form-control'
            ),
        ], className='col-md-4'),

        html.Div([
            html.Label('Filtrer par Niveau d\'expérience:', className='font-weight-bold', style={'color': 'white'}),
            dcc.Dropdown(
                id='dropdown-filtre-niveau-experience',
                # Construire les options pour le filtre par niveau d'expérience
                options=[{'label': str(exp), 'value': str(exp)} for exp in df_offres['EXPERIENCE_PROFESSIONNELLE'].unique()],
                value='Tous les niveaux d\'expérience',
                className='form-control'
            ),
        ], className='col-md-4'),
    ], className='row mb-4', style={'background-color': 'green'}),

    # Graphiques
    html.Div([
        dcc.Graph(id='graph-secteur-activite', className='col-md-6'),
        dcc.Graph(id='graph-entreprise', className='col-md-6'),
        dcc.Graph(id='graph-type-contrat', className='col-md-6'),
        dcc.Graph(id='graph-part-entreprises-par-type', className='col-md-6'),
        dcc.Graph(id='graph-part-entreprises-par-categorie', className='col-md-6'),
        dcc.Graph(id='graph-taille-moyenne-entreprises', className='col-md-6'),
    ], className='row', style={'background-color': 'white'}),
])

# Définir la logique des callbacks pour mettre à jour les graphiques en fonction des filtres
@app.callback(
    [Output('graph-secteur-activite', 'figure'),
     Output('graph-entreprise', 'figure'),
     Output('graph-type-contrat', 'figure'),
     Output('graph-part-entreprises-par-type', 'figure'),
     Output('graph-part-entreprises-par-categorie', 'figure'),
     Output('graph-taille-moyenne-entreprises', 'figure')],
    [Input('dropdown-filtre-secteur', 'value'),
     Input('dropdown-filtre-type-contrat', 'value'),
     Input('dropdown-filtre-diplome', 'value'),
     Input('dropdown-filtre-niveau-experience', 'value')]
)
def update_graphs(secteurs_activite, types_contrat, diplomes, niveaux_experience):
    # Filtrage du DataFrame en fonction des sélections
    df_filtered = df_offres

    # Définir les filtres à appliquer
    filters = {
        'BRANCHE_D_ACTIVITE': secteurs_activite,
        'TYPE_DE_CONTRAT_DU_POSTE': types_contrat,
        'DIPLOME': diplomes,
        'EXPERIENCE_PROFESSIONNELLE': niveaux_experience
    }

    # Appliquer les filtres
    for column, values in filters.items():
        if values and 'Tous' not in values:
            if isinstance(values, str):
                values = [values]  # Transformer en liste si c'est une chaîne de caractères unique
            df_filtered = df_filtered[df_filtered[column].isin(values)]

    # Graphique du nombre d'offres par secteur d'activité
    fig_secteur_activite = px.bar(df_filtered, x="BRANCHE_D_ACTIVITE", title="Nombre d'offres par secteur d'activité", color_discrete_sequence=['orange'])

    # Graphique du nombre d'offres par entreprise
    fig_entreprise = px.bar(df_filtered, x="SPECIALITE", title="Nombre d'offres par entreprise", color_discrete_sequence=['orange'])

    # Graphique du type de contrat proposé
    fig_type_contrat = px.bar(df_filtered, x="TYPE_DE_CONTRAT_DU_POSTE", title="Type de contrat proposé", color_discrete_sequence=['orange'])

    # Graphique pour la part des entreprises par type de contrat
    fig_part_entreprises_par_type = px.pie(df_filtered, names="TYPE_DE_CONTRAT_DU_POSTE", title="Part des entreprises par type de contrat", color_discrete_sequence=['orange'])

    # Graphique pour la part des entreprises par catégorie
    fig_part_entreprises_par_categorie = px.pie(df_filtered, names="EXPERIENCE_PROFESSIONNELLE", title="Part des entreprises par catégorie", color_discrete_sequence=['orange'])

    # Graphique pour la taille moyenne des entreprises par catégorie
    fig_taille_moyenne_entreprises = px.bar(df_filtered, x="EXPERIENCE_PROFESSIONNELLE", y=df_filtered.index, title="Taille moyenne des entreprises par catégorie", color_discrete_sequence=['orange'])

    return fig_secteur_activite, fig_entreprise, fig_type_contrat, fig_part_entreprises_par_type, fig_part_entreprises_par_categorie, fig_taille_moyenne_entreprises

# Exécuter l'application Dash
if __name__ == '__main__':
    app.run_server(debug=True, port=8058)



# Alerteemploi.net soucis sur leur site

In [3]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests

def collect_job_info(url):
    # Envoyer une requête GET à l'URL spécifiée
    response = requests.get(url)
    
    # Vérifier si la requête a réussi
    if response.status_code == 200:
        # Utiliser BeautifulSoup pour analyser le contenu HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Trouver tous les modules d'emploi
        job_modules = soup.find_all('div', class_='td_module_1')
        
        # Liste pour stocker les données des offres d'emploi
        job_data = []
        
        # Boucler à travers chaque module d'emploi et extraire les informations
        for module in job_modules:
            title = module.find('h3', class_='entry-title').text.strip()
            author = module.find('span', class_='td-post-author-name').text.strip()
            date_posted = module.find('time', class_='entry-date').text.strip()
            image_url = module.find('img', class_='entry-thumb')['src']
            post_url = module.find('h3', class_='entry-title').find('a')['href']
            
            # Ajouter les informations à la liste
            job_data.append({
                'Title': title,
                'Author': author,
                'Date Posted': date_posted,
                'Image URL': image_url,
                'Post URL': post_url
            })
        
        return job_data
    else:
        print("Failed to fetch the page.")
        return None

def scrape_additional_details(urls):
    # Initialiser une liste pour stocker les données
    job_data = []
    
    # Configurer Selenium pour s'exécuter en mode headless (sans ouvrir de fenêtre de navigateur)
    chrome_options = Options()
    chrome_options.add_argument("--headless")

    # Boucle à travers chaque URL
    for url in urls:
        try:
            # Initialiser le pilote Selenium
            driver = webdriver.Chrome(options=chrome_options)

            # Envoyer une requête GET en utilisant Selenium
            driver.get(url)

            # Attendre quelques secondes (ajustez selon les besoins)
            time.sleep(30)

            # Attendre que la page soit entièrement chargée
            WebDriverWait(driver, 10).until(
                lambda x: x.execute_script("return document.readyState === 'complete'")
            )

            # Récupérer le code source de la page après l'exécution de JavaScript
            page_source = driver.page_source

            # Fermer le pilote Selenium
            driver.quit()

            # Utiliser BeautifulSoup pour analyser le HTML
            soup = BeautifulSoup(page_source, 'html.parser')

            # Extraire les détails de l'offre d'emploi
            job_title_element = soup.find('h1', class_='entry-title')
            job_title = job_title_element.text.strip() if job_title_element else None

            author_container = soup.find('div', class_='td-post-author-name')
            author_element = author_container.find('a') if author_container else None
            author = author_element.text.strip() if author_element else None

            date_posted_element = soup.find('time', class_='entry-date')
            date_posted = date_posted_element['datetime'].strip() if date_posted_element else None

            views_element = soup.find('div', class_='td-post-views')
            views = views_element.find('span', class_='td-nr-views-19100').text.strip() if views_element and views_element.find('span', class_='td-nr-views-19100') else None

            image_url_element = soup.find('div', class_='td-post-featured-image')
            image_url = image_url_element.find('img')['src'] if image_url_element and image_url_element.find('img') else None

            # Ajouter les détails à la liste
            job_data.append({
                'Job Title': job_title,
                'Author': author,
                'Date Posted': date_posted,
                'Views': views,
                'Image URL': image_url,
                'URL': url
            })
        except Exception as e:
            print(f"Une erreur s'est produite pour l'URL {url}: {str(e)}")
            job_data.append({
                'Job Title': "",
                'Author': "",
                'Date Posted': "",
                'Views': "",
                'Image URL': "",
                'URL': url
            })

    return job_data

def alerte_emploi():
    # Liste des URLs des offres d'emploi
    job_listing_urls = [
        #"https://alerteemploi.net/toutes-les-offres/",
        "https://alerteemploi.net/category/emploi-stages/page/{}/".format(category) for category in range(52)
        # Ajoutez d'autres URLs au besoin
    ]

    # Récupérer les informations initiales des offres d'emploi
    initial_job_info = []
    for url in job_listing_urls:
        initial_job_info.extend(collect_job_info(url))

    # Extraire les URLs des offres d'emploi
    urls = [job['Post URL'] for job in initial_job_info]

    # Récupérer les détails supplémentaires des offres d'emploi en utilisant Selenium
    additional_job_info = scrape_additional_details(urls)

    # Fusionner les informations initiales et supplémentaires
    combined_job_info = []
    for initial_job, additional_job in zip(initial_job_info, additional_job_info):
        combined_job_info.append({**initial_job, **additional_job})

    # Créer un DataFrame
    df = pd.DataFrame(combined_job_info)
    return df

# Appeler la fonction alerte_emploi pour obtenir le DataFrame combiné
alerte_emploi_df = alerte_emploi()

# Afficher le DataFrame combiné
alerte_emploi_df

""


# Emploi_jeune.ci

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

def agence_emploi_jeunes():
    # Utilisation d'un en-tête pour éviter d'être bloqué
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Liste des URLs à scraper
    urls = ["https://agenceemploijeunes.ci/site/offres-emplois?page={}".format(category) for category in range(31)]

    # Listes pour stocker les données
    job_titles = []
    publication_dates = []
    application_deadlines = []
    locations = []
    job_descriptions = []
    job_types = []
    diploma_requirements = []
    url_lien = []

    # Loop à travers chaque URL
    for url in urls:
        # Envoyer une requête au site web
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')

        # Trouver les annonces d'emploi
        job_listings = soup.find_all('div', class_='post-bx')

        # Extract data from each job listing
        for job_listing in job_listings:
            url_lien.append(url)

            # Titre du poste
            job_title = job_listing.find('h4').text.strip()
            job_titles.append(job_title)

            # Dates de publication et de candidature
            date_info = job_listing.find_all('li', {'class': ''})
            if date_info:
                publication_date = date_info[0].text.replace('Publié le:', '').strip()
                application_deadline = date_info[1].text.replace('Date limite:', '').strip()
                publication_dates.append(publication_date)
                application_deadlines.append(application_deadline)

            # Localisation
            location = date_info[2].text.replace('ABENGOUROU', '').strip()
            locations.append(location)

            # Description du poste
            job_description = job_listing.find('p').text.strip()
            job_descriptions.append(job_description)

            # Type de poste
            job_type = job_listing.find('span', {'class': 'pull-right'}).text.strip()
            job_types.append(job_type)

            # Exigences en diplôme
            diploma_requirement = job_listing.find('div', {'class': 'salary-bx'}).text.strip()
            diploma_requirements.append(diploma_requirement)

    # Créer un DataFrame Pandas
    data = {
        'Job Title': job_titles,
        'Publication Date': publication_dates,
        'Application Deadline': application_deadlines,
        'Location': locations,
        'Job Description': job_descriptions,
        'Job Type': job_types,
        'Diploma Requirement': diploma_requirements,
        "URL": url_lien
    }

    df_agenceemploijeunes = pd.DataFrame(data)

    # Utiliser la méthode str.extract pour extraire la valeur après "Diplôme :"
    df_agenceemploijeunes['Diplome'] = df_agenceemploijeunes['Diploma Requirement'].str.extract(r'Diplôme :[ \t]([^\n\r])')

    # Scrapper les détails supplémentaires à partir des URL
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
    options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
    chrome_driver_path = 'C:\\Program Files (x86)\\chromedriver.exe'
    options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe" 
    #C:\Program Files\Google\Chrome\Application# Remplacez par l'emplacement réel de votre Chrome binary
    options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
    driver = webdriver.Chrome(options=options)

    # List to store job details
    all_job_details = []

    # Parcourir les liens
    for url in urls:
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        time.sleep(5)

        # Trouver toutes les offres d'emploi sur la page
        offer_links = soup.select('.post-bx h4 a')

        # Parcourir les liens d'offres
        for offer_link in offer_links:
            # Extraire l'URL de l'offre
            offer_url = offer_link.get('href')

            # Ajouter les détails à la liste
            all_job_details.append({'Offre_Link': offer_url, 'URL': url})

    # Fermer le navigateur à la fin
    driver.quit()

    # Créer un DataFrame avec les détails des offres d'emploi
    if all_job_details:
        all_job_details_df = pd.DataFrame(all_job_details)
    else:
        print("Aucun détail d'offre d'emploi trouvé.")

    # Fusionner les deux DataFrames sur la colonne 'URL'
    df_agenceemploi_jeunes = pd.merge(df_agenceemploijeunes, all_job_details_df, on='URL')
    df_agenceemploi_jeunes = df_agenceemploi_jeunes.drop_duplicates()
    df_agenceemploi_jeunes.reset_index(drop=True, inplace=True)

    # Création d'un dictionnaire pour stocker les données
    job_data = {
        'Job Title': [],
        'Location': [],
        'Reference': [],
        'Number of Positions': [],
        'Closing Date': [],
        'Diploma': [],
        'Job Type': [],
        'Experience': [],
        'Education Level': [],
        'Gender': [],
        'Job Description': [],
        'Offre_Link': []
    }

    # Loop through each URL
    for url in df_agenceemploi_jeunes["Offre_Link"]:
        offre_url = url  # Sauvegarder l'URL même en cas d'exception
        try:
            # Send a request to the website
            req = requests.get(url, headers=headers)
            req.raise_for_status()  # Raise an error for unsuccessful responses
            soup = BeautifulSoup(req.text, 'html.parser')

            # Extract job details
            job_details = soup.find('div', class_='widget_getintuch')

            if job_details:
                # Extract data from job details
                ul_element = job_details.find('ul')
                if ul_element:
                    details_list = ul_element.find_all('li')

                    # Initialize variables to store details
                    location = reference = num_positions = closing_date = diploma = job_type = experience = education_level = gender = None

                    # Iterate through details
                    for detail in details_list:
                        label = detail.find('strong')
                        value_span = detail.find('span', class_='text-black-light')

                        if label and value_span:
                            label_text = label.text.strip()
                            value_text = value_span.text.strip()

                            if 'Lieu de travail' in label_text:
                                location = value_text
                            elif 'Reference' in label_text:
                                reference = value_text
                            elif 'Nombre de poste' in label_text:
                                num_positions = value_text
                            elif 'Date de clôture' in label_text:
                                closing_date = value_text
                            elif 'Diplôme' in label_text:
                                diploma = value_text
                            elif 'Type de contrat' in label_text:
                                job_type = value_text
                            elif 'Expérience professionnelle' in label_text:
                                experience = value_text
                            elif 'Niveau d\'études' in label_text:
                                education_level = value_text
                            elif 'Sexe' in label_text:
                                gender = value_text

                    # Append extracted details to the dictionary
                    job_data['Location'].append(location)
                    job_data['Reference'].append(reference)
                    job_data['Number of Positions'].append(num_positions)
                    job_data['Closing Date'].append(closing_date)
                    job_data['Diploma'].append(diploma)
                    job_data['Job Type'].append(job_type)
                    job_data['Experience'].append(experience)
                    job_data['Education Level'].append(education_level)
                    job_data['Gender'].append(gender)

                # Extract job title and description
                job_title_element = soup.find('h3', {'class': 'title-head'})
                if job_title_element:
                    job_title = job_title_element.text.strip()
                    job_data['Job Title'].append(job_title)

                    job_description_info = soup.find('div', {'class': 'job-info-box'}).find('ul')
                    if job_description_info:
                        job_description_text = '\n'.join([li.text.strip() for li in job_description_info.find_all('li')])
                        job_data['Job Description'].append(job_description_text)
                    else:
                        job_data['Job Description'].append(None)
                else:
                    job_data['Job Title'].append(None)
                    job_data['Job Description'].append(None)
            else:
                job_data['Job Title'].append(None)
                job_data['Job Description'].append(None)

            # Append URL to the dictionary
            job_data['Offre_Link'].append(offre_url)

        except requests.exceptions.RequestException as e:
            print(f"An error occurred while accessing URL: {url}")
            print(e)
            # Ajouter l'URL même en cas d'exception
            job_data['Location'].append(None)
            job_data['Reference'].append(None)
            job_data['Number of Positions'].append(None)
            job_data['Closing Date'].append(None)
            job_data['Diploma'].append(None)
            job_data['Job Type'].append(None)
            job_data['Experience'].append(None)
            job_data['Education Level'].append(None)
            job_data['Gender'].append(None)
            job_data['Job Title'].append(None)
            job_data['Job Description'].append(None)
            job_data['Offre_Link'].append(offre_url)

    # Create DataFrame from the collected data
    df_jobs = pd.DataFrame(job_data)
    agenceemploi_jeunes_df = pd.merge(df_agenceemploi_jeunes, df_jobs, on='Offre_Link')
    agenceemploi_jeunes_dfagenceemploi_jeunes_df = agenceemploi_jeunes_df.drop_duplicates()
    agenceemploi_jeunes_df.reset_index(drop=True, inplace=True)
    equivalences = {
    "Job Title_x": "INTITULE_DU_POSTE",
    "Publication Date": "DATE_DE_PUBLICATION",
    "Application Deadline": None,
    "Location_x": "LIEU_DU_POSTE_DE_TRAVAIL",
    "Job Description_x": "DESCRIPTION_DU_POSTE",
    "Job Type_x": "TYPE_DE_CONTRAT_DU_POSTE",
    "Diploma Requirement": None,
    "URL": None,
    "Diplome": "DIPLOME_REQUIS",
    "Offre_Link": "SITE_WEB_DE_L_ENTREPRISE",
    # Colonnes sans équivalence
    "Location_y": None,
    "Reference": None,
    "Number of Positions": "NOMBRE_DE_POSTES_A_POURVOIR",
    "Closing Date": "DATE_D_EXPIRATION_DE_L_OFFRE",
    "Diploma": "DIPLOME",
    "Job Type_y": None,
    "Experience": None,
    "Education Level": "NIVEAU_D_ETUDES",
    "Gender": None,
    "Job Description_y": None}
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme
    agenceemploi_jeunes_df = renommer_colonnes(agenceemploi_jeunes_df, equivalences)
    
    return agenceemploi_jeunes_df
agenceemploi_jeunes_df=agence_emploi_jeunes()
agenceemploi_jeunes_df

In [ ]:
  equivalences = {
    "Offre_Link": "SITE_WEB_DE_L_ENTREPRISE",
    "Nom de l'entreprise": "RAISON_SOCIALE_DE_L_ENTREPRISE",
    "Secteur d'activité": "BRANCHE_D_ACTIVITE",
    "Lieu de travail": "LIEU_DU_POSTE_DE_TRAVAIL",
    "Date d'expiration": "DATE_D_EXPIRATION_DE_L_OFFRE",
    "Nombre de postes": "NOMBRE_DE_POSTES_A_POURVOIR",
    "Niveau de poste": None,
    "Niveau d'étude (diplôme)": "NIVEAU_D_ETUDES",
    "Type de contrat": "TYPE_DE_CONTRAT_DU_POSTE",
    "Provided Text": None,
    "Intitule": "INTITULE_DU_POSTE",
    "Entreprise": None,
    "Pays": "PAYS_DU_POSTE_DE_TRAVAIL",
    "Date": "DATE_DE_DEBUT_DE_L_OFFRE",
    "Niveau": None,
    "Experience_lettre": None,
    "url_Lien": "Offre_Link1"}

# rmo-jobcenter.com

In [2]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

def rmo_jobcenter():
    # Liste des URLs des pages d'offres d'emploi
    urls = [
        "https://rmo-jobcenter.com/fr/nos-offres-emploi.html",
        # Ajoutez d'autres URLs au besoin
    ]

    # Configurez Selenium pour s'exécuter en mode headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")

    # Liste pour stocker les données des offres d'emploi
    all_job_data = []

    # Base URL of the website
    base_url = "https://rmo-jobcenter.com"

    # Boucle à travers chaque URL
    for url in urls:
        # Initialisez le pilote Selenium
        driver = webdriver.Chrome(options=chrome_options)

        # Chargez la page avec Selenium
        driver.get(url)

        # Récupérez le code source de la page après l'exécution du JavaScript
        page_source = driver.page_source

        # Fermez le pilote Selenium
        driver.quit()

        # Utilisez BeautifulSoup pour analyser le HTML
        soup = BeautifulSoup(page_source, 'html.parser')

        # Trouvez la table contenant les offres d'emploi
        table = soup.find('table', class_='liste')

        # Vérifiez si la table est trouvée
        if table:
            # Liste pour stocker les données des offres d'emploi pour une URL spécifique
            job_data = []

            # Boucle à travers chaque ligne de la table (sauf la première qui contient les en-têtes)
            for row in table.find_all('tr')[1:]:
                # Extrayez les données de chaque colonne
                columns = row.find_all('td')
                date = columns[0].text.strip()
                filiale = columns[1].text.strip()
                fonction = columns[2].text.strip()
                secteur = columns[3].text.strip()
                reference = columns[4].text.strip()
                details_url = columns[5].find('a')['href']

                # Rendez l'URL absolue en la combinant avec l'URL de base.
                absolute_url = f"{base_url}/{details_url}"

                # Stockez les données dans un dictionnaire
                job_entry = {
                    'Date': date,
                    'Filiale': filiale,
                    'Fonction': fonction,
                    'Secteur': secteur,
                    'Référence / Statut': reference,
                    'Détails URL': absolute_url,
                    "URL": url
                }

                job_data.append(job_entry)

            # Ajoutez les données de cette URL à la liste globale
            all_job_data.extend(job_data)

    # Créez un DataFrame avec toutes les données extraites
    df_jobcenter = pd.DataFrame(all_job_data)

    # Si aucune donnée n'est extraite, retournez un DataFrame vide
    if df_jobcenter.empty:
        return df_jobcenter

    # Liste des URLs des pages d'offres d'emploi détaillées
    detail_urls = list(df_jobcenter['Détails URL'])

    # Liste pour stocker les données détaillées
    all_detail_data = []

    # Boucle à travers chaque URL détaillée
    for detail_url in detail_urls:
        # Initialisez le pilote Selenium
        driver = webdriver.Chrome(options=chrome_options)

        # Chargez la page avec Selenium
        driver.get(detail_url)

        # Récupérez le code source de la page après l'exécution du JavaScript
        page_source = driver.page_source

        # Fermez le pilote Selenium
        driver.quit()

        # Utilisez BeautifulSoup pour analyser le HTML
        soup = BeautifulSoup(page_source, 'html.parser')

        # Trouvez la div contenant les informations détaillées
        details_div = soup.find('div', {'id': 'content_articles'})

        # Vérifiez si la div est trouvée
        if details_div:
            # Extract details from the div
            job_title_element = details_div.find('div', {'id': 'h2_imprime'})
            job_title = job_title_element.text.strip() if job_title_element else None

            job_description_element = details_div.find('div', {'class': 'text-content'})
            job_description = job_description_element.text.strip() if job_description_element else None

            # Stockez les données dans un dictionnaire
            detail_entry = {
                'Job Title': job_title,
                'Job Description': job_description,
                'Détails URL': detail_url
                # Add more details as needed
            }

            # Ajoutez les données de cette URL à la liste globale
            all_detail_data.append(detail_entry)
        else:
            print("Aucune div avec l'ID 'content_articles' n'a été trouvée.")

    # Créez un DataFrame avec toutes les données extraites
    df_details = pd.DataFrame(all_detail_data)
    df_details[['Poste', 'Niveau', 'Sous Poste']] = df_details['Job Title'].str.split(' - ', expand=True)

    # Ajouter les listes existantes en tant que colonnes au DataFrame
    df_jobcenter=pd.merge(df_jobcenter,df_details, on='Détails URL')
    df_jobcenter=df_jobcenter.drop_duplicates()
    df_jobcenter.reset_index(drop=True, inplace=True)
    
    '''df_jobcenter['Job Title'] = df_details['Job Title']
    df_jobcenter['Job Description'] = df_details['Job Description']
    df_jobcenter['Job URL'] = df_details['Job URL']
    df_jobcenter['Poste'] = list(df_details['Poste'])
    df_jobcenter['Niveau'] = list(df_details['Niveau'])
    df_jobcenter['Sous Poste'] =  list(df_details['Sous Poste'])'''
    equivalences = {
    "Date": "DATE",
    "Filiale": "FILIALE",
    "Fonction": "FONCTION",
    "Secteur": "SECTEUR",
    "Référence / Statut": "REFERENCE_STATUT",
    "Détails URL": "DETAILS_URL",
    "Job Title": "INTITULE_DU_POSTE",
    "Job Description": "DESCRIPTION_DU_POSTE",
    "Job URL": "URL_DU_POSTE",
    "Poste": "POSTE",
    "Niveau": "DIPLOME",
    "Sous Poste": "SOUS_POSTE"}
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme
    
    df_jobcenter = renommer_colonnes(df_jobcenter, equivalences)
    # Réorganiser les colonnes selon vos besoins

    return df_jobcenter

# Appeler la fonction rmo_jobcenter pour obtenir le DataFrame combiné
rmo_jobcenter_df = rmo_jobcenter()

# Afficher le DataFrame combiné
 
rmo_jobcenter_df[['candidature','Data_cloture']] = rmo_jobcenter_df['SOUS_POSTE'].str.split('-', n=1, expand=True)
rmo_jobcenter_df.drop(columns=['candidature'], inplace=True)
# Fonction de nettoyage pour extraire uniquement la date
def clean_date(date_str):
    return date_str.split('expire le ')[1].strip() if isinstance(date_str, str) else date_str

# Appliquer la fonction de nettoyage à toute la colonne 'Data_cloture'
rmo_jobcenter_df['Data_cloture'] = rmo_jobcenter_df['Data_cloture'].apply(clean_date)

# Afficher le DataFrame après nettoyage
rmo_jobcenter_df

,DATE,FILIALE,FONCTION,SECTEUR,REFERENCE_STATUT,DETAILS_URL,URL,INTITULE_DU_POSTE,DESCRIPTION_DU_POSTE,POSTE,DIPLOME,SOUS_POSTE,Data_cloture
0,18/03/2024,,Business Development Manager (H/F),Agriculture,Réf: #782185Réception candidatureexpire le 18/...,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Business Development Manager (H/F) - CDI - Réc...,"Le Poste\nRMO\nImplanté en Afrique,\nRMO vous ...",Business Development Manager (H/F),CDI,Réception candidature - expire le 18/04/2024,18/04/2024
1,18/03/2024,,Chef Comptable et Financier Bilingue (H/F),Industrie / Production/ Mines,Réf: #205874Réception candidatureexpire le 18/...,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Chef Comptable et Financier Bilingue (H/F) - C...,"Le Poste\nRMO\nImplanté en Afrique,\nRMO vous ...",Chef Comptable et Financier Bilingue (H/F),CDI,Réception candidature - expire le 18/04/2024,18/04/2024
2,15/03/2024,,Ingénieur QOS/Reporting Zones Huawei et Nokia ...,Informatique / Télécom,Réf: #492687Réception candidatureexpire le 25/...,https://rmo-jobcenter.com/fr/mali/offres-emplo...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Ingénieur QOS/Reporting Zones Huawei et Nokia ...,Le PosteRMO Mali recrute un(e) Ingénieur\nQOS ...,Ingénieur QOS/Reporting Zones Huawei et Nokia ...,Autre,Réception candidature - expire le 25/03/2024,25/03/2024
3,15/03/2024,,Technicien de Supervision (H/F),Informatique / Télécom,Réf: #121240Réception candidatureexpire le 25/...,https://rmo-jobcenter.com/fr/mali/offres-emplo...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Technicien de Supervision (H/F) - Autre - Réce...,Le PosteRMO Mali recrute un(e) Technicien(e) ...,Technicien de Supervision (H/F),Autre,Réception candidature - expire le 25/03/2024,25/03/2024
4,07/03/2024,,Chauffeurs Poids Lourd (H/F),Industrie / Production/ Mines,Réf: #200021Réception candidatureexpire le 29/...,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Chauffeurs Poids Lourd (H/F) - CDD - Réceptio...,Le PosteLe chauffeur poids lourd aura pour pri...,Chauffeurs Poids Lourd (H/F),CDD,Réception candidature - expire le 29/05/2024,29/05/2024
5,04/03/2024,,Chauffeur-Vendeur Terrain (H/F),Industrie / Production/ Mines,Réf: #645373Réception candidatureexpire le 05/...,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Chauffeur-Vendeur Terrain (H/F) - CDD - Récep...,Le PosteClient : Agroalimentaire.MissionSous l...,Chauffeur-Vendeur Terrain (H/F),CDD,Réception candidature - expire le 05/05/2024,05/05/2024
6,26/02/2024,,Receptionniste (H/F),Immobilier,Réf: #489229Réception candidatureexpire le 26/...,https://rmo-jobcenter.com/fr/senegal/offres-em...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Receptionniste (H/F) - CDI - Réception candida...,Le PosteNous recherchons un Réceptionniste H/F...,Receptionniste (H/F),CDI,Réception candidature - expire le 26/03/2024,26/03/2024
7,19/02/2024,,Chef de rayon (H/F),Commercial / Vente/ Distribution,Réf: #516522Réception candidatureexpire le 30/...,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Chef de rayon (H/F) - CDI - Réception candidat...,"Le PosteRMOImplanté en Afrique, RMO vous propo...",Chef de rayon (H/F),CDI,Réception candidature - expire le 30/04/2024,30/04/2024
8,13/02/2024,,Directeur Magasin (H/F),Commercial / Vente/ Distribution,Réf: #572498Réception candidatureexpire le 30/...,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Directeur Magasin (H/F) - CDI - Réception cand...,"Le PosteRMOImplanté en Afrique, RMO vous propo...",Directeur Magasin (H/F),CDI,Réception candidature - expire le 30/04/2024,30/04/2024
9,13/02/2024,,Manager Boutique (H/F),Commercial / Vente/ Distribution,Réf: #356936Réception candidatureexpire le 13/...,https://rmo-jobce

In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Liste des URLs à analyser
detail_urls = list(rmo_jobcenter_df['DETAILS_URL'])

# Initialiser une liste pour stocker les données extraites de chaque URL
data_list = []

# Boucle sur chaque URL dans la liste
for detail_url in detail_urls:
    try:
        # Envoyer une requête GET pour récupérer le contenu de la page
        response = requests.get(detail_url)

        # Vérifier si la requête a réussi (code 200)
        if response.status_code == 200:
            # Extraire le contenu HTML
            html = response.text

            # Analyser la structure HTML avec BeautifulSoup
            soup = BeautifulSoup(html, 'html.parser')

            # Extraire les informations de la page
            informations = {'DETAILS_URL': detail_url}

            # Extraire le poste s'il existe
            poste_element = soup.find('h4', text='Le Poste')
            if poste_element:
                poste = poste_element.find_next('div').get_text(strip=True)
                informations['Le Poste'] = poste

            # Extraire le profil du candidat s'il existe
            profil_element = soup.find('h4', text='Profil du Candidat')
            if profil_element:
                profil_candidat = profil_element.find_next('div').get_text(strip=True)
                informations['Profil du Candidat'] = profil_candidat

            # Extraire les critères client s'ils existent
            critere_client_element = soup.find('h6', class_='le_grd_titre', text='Critères Client')
            if critere_client_element:
                critere_client_rows = critere_client_element.find_next('tbody').find_all('tr')
                critere_client = {row.find('th').get_text(strip=True): row.find('td').get_text(strip=True) for row in critere_client_rows}
                informations.update(critere_client)
            else:
                print(f"Les critères client n'ont pas été trouvés sur la page : {detail_url}")

            # Extraire les autres informations
            autres_informations_element = soup.find('h4', text='Autres Informations')
            if autres_informations_element:
                autres_informations_rows = autres_informations_element.find_next('tbody').find_all('tr')
                autres_informations = {row.find('th').get_text(strip=True): row.find('td').get_text(strip=True) for row in autres_informations_rows}
                informations.update(autres_informations)

            # Ajouter les informations extraites à la liste
            data_list.append(informations)
        else:
            print(f"La requête GET a échoué pour l'URL : {detail_url}")

    except Exception as e:
        print(f"Une erreur s'est produite lors de l'analyse de l'URL {detail_url}: {str(e)}")

# Créer un DataFrame pandas à partir des informations extraites
data = pd.DataFrame(data_list)

# Afficher le DataFrame

data['Postes Disponibles :']=data['Postes Disponibles :'].str.extract(r"([0-9,]+)")
data[['annee_min', 'annee_max']] = data['Années d\'Expérience :'].str.extract(r'(\d+)\-\s*(\d+)\s*ans?')

# Remplir les valeurs manquantes dans annee_min avec les valeurs extraites de Année d'Expérience Requise
data['annee_min'].fillna(data['Années d\'Expérience :'], inplace=True)  # Remplissage des valeurs manquantes dans la colonne 'annee_min' avec les valeurs d'origine
data['annee_max'].fillna(data['Années d\'Expérience :'], inplace=True)  # Remplissage des valeurs manquantes dans la colonne 'annee_max' avec les valeurs d'origine
# Extraire uniquement les chiffres de chaque colonne
data['annee_min'] = data['annee_min'].str.extract(r"([0-9,]+)")  # Extraction des chiffres de la colonne 'annee_min'
data['annee_max'] = data['annee_max'].str.extract(r"([0-9,]+)")  # Extraction des chiffres de la colonne 'annee_max'
data[["VILLE_DU_POSTE_DE_TRAVAIL", "DU_POSTE_DE_TRAVAIL","PAYS_DU_POSTE_DE_TRAVAIL"]] = data['Localisation :'].str.split('-', expand=True)

for i in range(len(data["PAYS_DU_POSTE_DE_TRAVAIL"])):
    if data["PAYS_DU_POSTE_DE_TRAVAIL"][i] is None :
        data["PAYS_DU_POSTE_DE_TRAVAIL"][i]=data["DU_POSTE_DE_TRAVAIL"][i]
    if data["PAYS_DU_POSTE_DE_TRAVAIL"][i] != data["DU_POSTE_DE_TRAVAIL"][i]:
        ville = str(data["VILLE_DU_POSTE_DE_TRAVAIL"][i])  # Convertir en str pour éviter les problèmes de type
        du_poste = str(data["DU_POSTE_DE_TRAVAIL"][i])
        data["VILLE_DU_POSTE_DE_TRAVAIL"][i] = ville + " - " + du_poste    

#Supprimer la colonne 
data.drop(columns=["DU_POSTE_DE_TRAVAIL"], inplace=True)
# Affichage du DataFrame résultant
data
  # Affichage du DataFrame résultant


Les critères client n'ont pas été trouvés sur la page : https://rmo-jobcenter.com/fr/cote-d-ivoire/offres-emploi/industrie-production-mat-1eres/3365-business-development-manager.html
Les critères client n'ont pas été trouvés sur la page : https://rmo-jobcenter.com/fr/cote-d-ivoire/offres-emploi/industrie-production-mat-1eres/3366-chef-comptable-financier-bilingue.html
Les critères client n'ont pas été trouvés sur la page : https://rmo-jobcenter.com/fr/mali/offres-emploi/ntic/3363-ingenieur-qos-reporting-zones.html
Les critères client n'ont pas été trouvés sur la page : https://rmo-jobcenter.com/fr/mali/offres-emploi/ntic/3364-technicien-supervision.html
Les critères client n'ont pas été trouvés sur la page : https://rmo-jobcenter.com/fr/senegal/offres-emploi/commerce-vente-distribution/3359-receptionniste.html
Les critères client n'ont pas été trouvés sur la page : https://rmo-jobcenter.com/fr/togo/offres-emploi/services/2798-responsable-commercial.html


,DETAILS_URL,Le Poste,Profil du Candidat,Filiale RMO :,N° de l'Offre :,Secteur d'Activité :,Fonction :,Type de Contrat :,Localisation :,Rémunération :,Postes Disponibles :,Niveau d'Etudes :,Années d'Expérience :,Langues :,Informatique :,Avantages :,annee_min,annee_max,VILLE_DU_POSTE_DE_TRAVAIL,PAYS_DU_POSTE_DE_TRAVAIL
0,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,"RMOImplanté en Afrique,\r\nRMO vous propose de...","ProfilTitulaire dun BAC+4/5\r\nen Agronomie, ...",RMO Côte d'Ivoire,#782185,Agriculture,Business Development Manager,CDI,Abidjan-Côte d'Ivoire,Salaire fixe,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abidjan,Côte d'Ivoire
1,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,"RMOImplanté en Afrique,\r\nRMO vous propose de...","ProfilTitulaire dun BAC+5\r\nen Comptabilité,...",RMO Côte d'Ivoire,#205874,Industrie / Production/ Mines,Chef Comptable et Financier Bilingue,CDI,Abidjan-Côte d'Ivoire,Salaire fixe,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abidjan,Côte d'Ivoire
2,https://rmo-jobcenter.com/fr/mali/offres-emplo...,Profil du CandidatVotre profil :Formation souh...,"Le Dépôt de Candidature, la Phase de Sélection...",RMO Mali,#492687,Informatique / Télécom,Ingénieur QOS/Reporting Zones Huawei et Nokia,Autre,Bamako- kati-Mali,Salaire fixe + possibilités variables,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bamako - kati,Mali
3,https://rmo-jobcenter.com/fr/mali/offres-emplo...,Profil du CandidatVotre profil :Formation souh...,"Le Dépôt de Candidature, la Phase de Sélection...",RMO Mali,#121240,Informatique / Télécom,Technicien de Supervision,Autre,Bamako- kati-Mali,Salaire fixe + possibilités variables,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bamako - kati,Mali
4,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,Le chauffeur poids lourd aura pour principales...,"Le Dépôt de Candidature, la Phase de Sélection...",RMO Côte d'Ivoire,#200021,Industrie / Production/ Mines,Chauffeurs Poids Lourd,CDD,Abidjan-Côte d'Ivoire,Salaire fixe,2,Bac,1-2 an(s),Français,NaN,NaN,1,2,Abidjan,Côte d'Ivoire
5,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,Client : Agroalimentaire.,"Le Dépôt de Candidature, la Phase de Sélection...",RMO Côte d'Ivoire,#645373,Industrie / Production/ Mines,Chauffeur-Vendeur Terrain,CDD,Abidjan-Côte d'Ivoire,Salaire fixe,4,Bac,1-2 an(s),Français,NaN,NaN,1,2,Abidjan,Côte d'Ivoire
6,https://rmo-jobcenter.com/fr/senegal/offres-em...,,Autres certifications ou diplômes valorisés.,RMO Sénégal,#489229,Immobilier,Receptionniste,CDI,Dakar-Sénégal,Salaire fixe,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dakar,Sénégal
7,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,RMO,,RMO Côte d'Ivoire,#516522,Commercial / Vente/ Distribution,Chef de rayon,CDI,Abidjan-Côte d'Ivoire,Salaire fixe,4,Bac+2,1-2 an(s),Français,Word - Excel,NaN,1,2,Abidjan,Côte d'Ivoire
8,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,"RMOImplanté en Afrique, RMO vous propose des s...",,RMO Côte d'Ivoire,#572498,Commercial / Vente/ Distribution,Directeur Magasin,CDI,Abidjan-Côte d'Ivoire,Salaire fixe,1,Bac+5,6-8 an(s),Français,Word - Excel - PowerPoint,NaN,6,8,Abidjan,Côte d'Ivoire
9,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,Profil du CandidatProfilTitulaire dun Bac+2/5...,"Le Dépôt de Candidature, la Phase de Sélection...",RMO Côte d'Ivoire,#356936,Commercial / Vente/ Distribution,Manager Boutique,CDI,Abidjan-Côte d'Ivoire,Salaire fixe,1,Bac+2,6-8 an(s),NaN,Word - Excel - PowerPoint,NaN,6,8,Abidjan,Côte d'Ivoire


In [6]:
rmo_jobcenter_df = pd.merge(data,rmo_jobcenter_df, on='DETAILS_URL')
rmo_jobcenter_df = rmo_jobcenter_df.drop_duplicates()
rmo_jobcenter_df.reset_index(drop=True, inplace=True)
rmo_jobcenter_df

,DETAILS_URL,Le Poste,Profil du Candidat,Filiale RMO :,N° de l'Offre :,Secteur d'Activité :,Fonction :,Type de Contrat :,Localisation :,Rémunération :,...,FONCTION,SECTEUR,REFERENCE_STATUT,URL,INTITULE_DU_POSTE,DESCRIPTION_DU_POSTE,POSTE,DIPLOME,SOUS_POSTE,Data_cloture
0,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,"RMOImplanté en Afrique,\r\nRMO vous propose de...","ProfilTitulaire dun BAC+4/5\r\nen Agronomie, ...",RMO Côte d'Ivoire,#782185,Agriculture,Business Development Manager,CDI,Abidjan-Côte d'Ivoire,Salaire fixe,...,Business Development Manager (H/F),Agriculture,Réf: #782185Réception candidatureexpire le 18/...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Business Development Manager (H/F) - CDI - Réc...,"Le Poste\nRMO\nImplanté en Afrique,\nRMO vous ...",Business Development Manager (H/F),CDI,Réception candidature - expire le 18/04/2024,18/04/2024
1,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,"RMOImplanté en Afrique,\r\nRMO vous propose de...","ProfilTitulaire dun BAC+5\r\nen Comptabilité,...",RMO Côte d'Ivoire,#205874,Industrie / Production/ Mines,Chef Comptable et Financier Bilingue,CDI,Abidjan-Côte d'Ivoire,Salaire fixe,...,Chef Comptable et Financier Bilingue (H/F),Industrie / Production/ Mines,Réf: #205874Réception candidatureexpire le 18/...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Chef Comptable et Financier Bilingue (H/F) - C...,"Le Poste\nRMO\nImplanté en Afrique,\nRMO vous ...",Chef Comptable et Financier Bilingue (H/F),CDI,Réception candidature - expire le 18/04/2024,18/04/2024
2,https://rmo-jobcenter.com/fr/mali/offres-emplo...,Profil du CandidatVotre profil :Formation souh...,"Le Dépôt de Candidature, la Phase de Sélection...",RMO Mali,#492687,Informatique / Télécom,Ingénieur QOS/Reporting Zones Huawei et Nokia,Autre,Bamako- kati-Mali,Salaire fixe + possibilités variables,...,Ingénieur QOS/Reporting Zones Huawei et Nokia ...,Informatique / Télécom,Réf: #492687Réception candidatureexpire le 25/...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Ingénieur QOS/Reporting Zones Huawei et Nokia ...,Le PosteRMO Mali recrute un(e) Ingénieur\nQOS ...,Ingénieur QOS/Reporting Zones Huawei et Nokia ...,Autre,Réception candidature - expire le 25/03/2024,25/03/2024
3,https://rmo-jobcenter.com/fr/mali/offres-emplo...,Profil du CandidatVotre profil :Formation souh...,"Le Dépôt de Candidature, la Phase de Sélection...",RMO Mali,#121240,Informatique / Télécom,Technicien de Supervision,Autre,Bamako- kati-Mali,Salaire fixe + possibilités variables,...,Technicien de Supervision (H/F),Informatique / Télécom,Réf: #121240Réception candidatureexpire le 25/...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Technicien de Supervision (H/F) - Autre - Réce...,Le PosteRMO Mali recrute un(e) Technicien(e) ...,Technicien de Supervision (H/F),Autre,Réception candidature - expire le 25/03/2024,25/03/2024
4,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,Le chauffeur poids lourd aura pour principales...,"Le Dépôt de Candidature, la Phase de Sélection...",RMO Côte d'Ivoire,#200021,Industrie / Production/ Mines,Chauffeurs Poids Lourd,CDD,Abidjan-Côte d'Ivoire,Salaire fixe,...,Chauffeurs Poids Lourd (H/F),Industrie / Production/ Mines,Réf: #200021Réception candidatureexpire le 29/...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Chauffeurs Poids Lourd (H/F) - CDD - Réceptio...,Le PosteLe chauffeur poids lourd aura pour pri...,Chauffeurs Poids Lourd (H/F),CDD,Réception candidature - expire le 29/05/2024,29/05/2024
5,https://rmo-jobcenter.com/fr/cote-d-ivoire/off...,Client : Agroalimentaire.,"Le Dépôt de Candidature, la Phase de Sélection...",RMO Côte d'Ivoire,#645373,Industrie / Production/ Mines,Chauffeur-Vendeur Terrain,CDD,Abidjan-Côte d'Ivoire,Salaire fixe,...,Chauffeur-Vendeur Terrain (H/F),Industrie / Production/ Mines,Réf: #645373Réception candidatureexpire le 05/...,https://rmo-jobcenter.com/fr/nos-offres-emploi...,Chauffeur-Vendeur Terrain (H/F) - CDD - Récep...,Le PosteClient : Agroalimentaire.Mis

# projobivoire

In [7]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def projobivoire():
    def extract_text(element, tag_name=None):
        if element and tag_name:
            tag = element.find(tag_name)
            return tag.text.strip() if tag else ""
        return ""

    def clean_text(text):
        return text.replace('\r\n', '').replace('\xa0', '')

    def scrape_projobivoire_page(page_url):
        job_data_list = []

        for url in page_url:
            try:
                response = requests.get(url, timeout=500)
                response.raise_for_status()
            except requests.exceptions.RequestException as e:
                    print(f"Erreur de connexion à {url} : {e}")
                    continue

            soup = BeautifulSoup(response.text, 'html.parser')

            job_items = soup.find_all('div', class_='loop-item-wrap list')

            if not job_items:
                print(f"Aucun élément de travail trouvé pour l'URL : {url}")
                continue

            for job_item in job_items:
                title_tag = job_item.find('h3', class_='loop-item-title')
                title = extract_text(title_tag, 'a')

                job_type_tag = job_item.find('span', class_='job-type')
                job_type = extract_text(job_type_tag, 'span')

                job_date_posted = soup.find('span', class_='job-date__posted').text.strip()

                job_date_closing_tag = soup.find('span', class_='job-date__closing')
                job_date_closing = job_date_closing_tag.text.strip() if job_date_closing_tag else ""

                job_date_closing = job_date_closing.lstrip('-').strip()

                category_tag = job_item.find('span', class_='job-category')
                category = extract_text(category_tag, 'a')

                # Ajout de ces lignes pour extraire l'URL de l'e-mail
                email_url_tag = job_item.find('span', class_='noo-tool-email-job')
                email_url = email_url_tag['data-url'] if email_url_tag else ""

                data = {
                    'Title': title,
                    'Type': job_type,
                    'DatePosted': job_date_posted,
                    'DateClosing': job_date_closing,
                    'Category': category,
                    'EmailURL': email_url,
                    'URL': url
                }

                job_data_list.append(data)

        return job_data_list

    # Liste des URL de pages avec plusieurs offres d'emploi
    page_urls = ["https://projobivoire.com/page/{}/".format(category) for category in range(546)]

    # Scrape des détails de chaque offre d'emploi sur les pages
    job_data_list = scrape_projobivoire_page(page_urls)

    # Création d'un DataFrame à partir de la liste des données d'emploi
    df_projobivoire = pd.DataFrame(job_data_list)

    # Affichage du DataFrame

    # Définition de l'équivalence entre les variables
    equivalences = {
        "Title": "INTITULE_DU_POSTE",
        "Type": "TYPE_DE_CONTRAT_DU_POSTE",
        "DatePosted": "DATE_DE_PUBLICATION",
        "DateClosing": "DATE_D_EXPIRATION_DE_L_OFFRE",
        "Category": "CATEGORIE",
        "EmailURL": None,
        "URL": "SITE_WEB_DE_L_ENTREPRISE"
    }

    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme

    # Renommer les colonnes du DataFrame
    df_projobivoire = renommer_colonnes(df_projobivoire, equivalences)

    # Afficher le DataFrame avec les colonnes renommées
    return df_projobivoire

# Appeler la fonction projobivoire pour obtenir le DataFrame des offres d'emploi
df_projobivoire = projobivoire()

# Afficher le DataFrame
df_projobivoire

,INTITULE_DU_POSTE,TYPE_DE_CONTRAT_DU_POSTE,DATE_DE_PUBLICATION,DATE_D_EXPIRATION_DE_L_OFFRE,CATEGORIE,EmailURL,SITE_WEB_DE_L_ENTREPRISE
0,FORMATIONS INDUSTRIELLES100% PRATIQUES,Emploi,25 mars 2024,25 mars 2024,Formation,https://projobivoire.com/jobs/formations-indus...,https://projobivoire.com/page/0/
1,Consultant(e) pour la réalisation de la Cartog...,Consultance,25 mars 2024,25 mars 2024,Cartographie,https://projobivoire.com/jobs/consultante-pour...,https://projobivoire.com/page/0/
2,Responsable Informatique Microfinance-Sénégal,Emploi,25 mars 2024,25 mars 2024,Informatique,https://projobivoire.com/jobs/responsable-info...,https://projobivoire.com/page/0/
3,Chef Comptable Microfinance-Sénégal,Emploi,25 mars 2024,25 mars 2024,Finances/Comptabilité,https://projobivoire.com/jobs/chef-comptable-m...,https://projobivoire.com/page/0/
4,Directeur Résidence Hotellière–Korhogo-Côte d’...,Emploi,25 mars 2024,25 mars 2024,Tourisme/Loisirs,https://projobivoire.com/jobs/directeur-reside...,https://projobivoire.com/page/0/
...,...,...,...,...,...,...,...
9277,L’ONECI recrute plusieurs profils,,18 août 2021,27 août 2021,,https://projobivoire.com/jobs/oneci-recrute-pl...,https://projobivoire.com/page/545/
9278,La SIFAAP recrute plusieurs profils,Emploi,18 août 2021,27 août 2021,,https://projobivoire.com/jobs/la-sifaap-recrut...,https://projobivoire.com/page/545/
9279,ADMINISTRATIVE TALENTS MANAGER,Emploi,18 août 2021,27 août 2021,Ressources Humaines,https://projobivoire.com/jobs/administrative-t...,https://projobivoire.com/page/545/
9280,Apave recrute Technicien chargé d’inspection d...,Emploi,18 août 2021,27 août 2021,Electrotechnique/Electricité,https://projobivoire.com/jobs/apave-recrute/,https://projobivoire.com/page/545/


# ci.talent

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def talent_ci():
    def extract_text(element, tag_name=None):
        tag = element.find(tag_name)
        return tag.text.strip() if tag else ""

    def clean_text(text):
        return text.replace('\r\n', '').replace('\xa0', '')

    def scrape_talent_com(url):
        try:
            response = requests.get(url, timeout=500)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion à {url} : {e}")
            return pd.DataFrame()

        soup = BeautifulSoup(response.text, 'html.parser')

        job_wrappers = soup.find_all('div', class_='card card__job')

        data_list = []

        for wrapper in job_wrappers:
            title_tag = wrapper.find('h2', class_='card__job-title')
            title = extract_text(title_tag, 'a')
            
            employer_location_tag = wrapper.find('div', class_='card__job-empnameLocation')
            #employer = extract_text(employer_location_tag.find('div', class_='card__job-location'))  # Extract location from the inner div
            location= extract_text(employer_location_tag, 'div')

            employer_location_tag = wrapper.find('div', class_='card__job-empname-label')
            
            # Extracting employer and description from the div
            employer = employer_location_tag.text.strip() if employer_location_tag else None  # Extract location from the inner div
            
            description_tag = wrapper.find('div', class_='card__job-snippet-logo')
            description = clean_text(extract_text(description_tag, 'p'))

            data_list.append({
                'Title': title,
                'Location': location,
                'Employer': employer,
                'Description': description,
                'URL': url
            })

        df = pd.DataFrame(data_list)
        return df

    # List of URLs for talent.com jobs
    urls = [
         "https://ci.talent.com/jobs?l=Abidjan%2C+Abidjan&radius=15&p={}&k=&context=serp_pagination".format(category) for category in range(8)
        # Add more URLs as needed "https://ci.talent.com/jobs",
    ]

    # Initialize an empty DataFrame to store the results
    ci_talent = pd.DataFrame()

    # Scrape job information for each URL and concatenate the results
    for url in urls:
        df = scrape_talent_com(url)
        ci_talent = pd.concat([ci_talent, df], ignore_index=True)

    # Définition de l'équivalence entre les variables
    equivalences = {
        "Title": "INTITULE_DU_POSTE",
        "Location": "LIEU_DU_POSTE_DE_TRAVAIL",
        "Employer": "RAISON_SOCIALE_DE_L_ENTREPRISE",
        "Description": "DESCRIPTION_DU_POSTE",
        "URL": "SITE_WEB_DE_L_ENTREPRISE"
    }

    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme

    # Renommer les colonnes du DataFrame
    ci_talent = renommer_colonnes(ci_talent, equivalences)

    # Afficher le DataFrame avec les colonnes renommées
    return ci_talent

# Appeler la fonction talent_ci pour obtenir le DataFrame des offres d'emploi
df_talent_ci = talent_ci()

# Afficher le DataFrame
df_talent_ci

,INTITULE_DU_POSTE,LIEU_DU_POSTE_DE_TRAVAIL,RAISON_SOCIALE_DE_L_ENTREPRISE,DESCRIPTION_DU_POSTE,SITE_WEB_DE_L_ENTREPRISE
0,Country Technical Sales Manager – West Africa ...,"Abidjan, Abidjan",Claire Joster,PresentaciónClaire Joster is a headhunting fir...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
1,Analyste Financier,"Abidjan, Abidjan Autonomous District",Coface,Entreprise. 70 ans d'expérience et un maillage...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
2,Teachers.,"Abidjan, Abidjan",Enko Education,Enko Education is recruiting teachers to drive...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
3,Investment Manager or Principal,"Abidjan, Abidjan",Seedstars,About the Company Seedstars Africa Ventures (S...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
4,Fulfillment manager,"Abidjan, District Autonome d'Abidjan",Lapaire Group,"At Lapaire, we believe that everyone has the r...",https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
...,...,...,...,...,...
155,Manager Juridique et Fiscal F/H,"Abidjan, Lagunes",Taj,Tous nos postes sont ouverts au télétravailRej...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
156,Chef de Projet Tuyauterie,"Abidjan, Abidjan",ORTEC Group,"À propos de nous Présent dans 25 pays, le Grou...",https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
157,VIE Ingénieur travaux travaux souterrains - Af...,"Abidjan, Abidjan Autonomous District",Bessac,"En plein développement, la filiale Bessac Afri...",https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...
158,Adjoint aux actions civilo-militaires au sein ...,"Abidjan, Lagunes District",Ministère des Armées,Cette offre est publiée sous réserve des évolu...,https://ci.talent.com/jobs?l=Abidjan%2C+Abidja...


# worldbankgroup mondiale

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def mondiale_ci():
    # Fonction pour extraire les données d'une page
    def scrape_page(url):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Lèvera une exception si la requête a échoué
            soup = BeautifulSoup(response.content, "html.parser")
            job_rows = soup.find("table", class_="results-table").find("tbody").find_all("tr")

            # Listes pour stocker les données de cette page
            job_titles = []
            locations = []
            job_families = []
            deadlines = []

            # Parcourir chaque ligne du tableau et extraire les informations nécessaires
            for row in job_rows:
                # Récupérer le titre de l'emploi
                job_title = row.find("a").text.strip()
                job_titles.append(job_title)

                # Récupérer l'emplacement
                location = row.find_all("td")[1].text.strip()
                locations.append(location)

                # Récupérer la famille d'emploi
                job_family = row.find_all("td")[2].text.strip()
                job_families.append(job_family)

                # Récupérer la date limite d'application
                deadline = row.find_all("td")[3].text.strip()
                deadlines.append(deadline)

            # Retourner les données de cette page sous forme de DataFrame
            data = {
                "Job Title": job_titles,
                "Location": locations,
                "Job Family": job_families,
                "Deadline": deadlines
            }
            return pd.DataFrame(data)
        except requests.exceptions.RequestException as e:
            print("Une erreur s'est produite lors de la requête:", e)
            return pd.DataFrame()  # Retourner un DataFrame vide en cas d'erreur

    # URL de la première page
    base_url = "https://worldbankgroup.csod.com/ats/careersite/search.aspx?site=1&c=worldbankgroup&sid=%5e%5e%5eFLGscZMYY2RrwVaMR%2ftHYw%3d%3d"

    # Créer une liste pour stocker les DataFrames de chaque page
    dfs = []

    # Extraire les données de la première page
    dfs.append(scrape_page(base_url))

    # Trouver le nombre total de pages
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, "html.parser")
    pagination_span = soup.find("span", class_=re.compile(r"\btext\b"), text=re.compile(r"\d+"))
    if pagination_span:
        num_pages = int(pagination_span.text.strip())
    else:
        num_pages = 1

    # Boucle à travers chaque page et extraire les données
    for page_num in range(1, num_pages + 1):
        page_url = f"{base_url}&pg={page_num}"
        df = scrape_page(page_url)
        if not df.empty:  # Vérifier si le DataFrame n'est pas vide
            dfs.append(df)

    # Concaténer tous les DataFrames en un seul
    df_mondiale = pd.concat(dfs, ignore_index=True)

    # Définition de l'équivalence entre les variables
    equivalences = {
        "Job Title": "INTITULE_DU_POSTE",
        "Location": "LIEU_DU_POSTE_DE_TRAVAIL",
        "Job Family": "SECTEUR",
        "Deadline": "DATE_D_EXPIRATION_DE_L_OFFRE"
    }

    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme

    # Renommer les colonnes du DataFrame
    df_mondiale = renommer_colonnes(df_mondiale, equivalences)

    # Afficher le DataFrame avec les colonnes renommées
    return df_mondiale

# Appel de la fonction principale

mondiale_ci=mondiale_ci()
mondiale_ci

,INTITULE_DU_POSTE,LIEU_DU_POSTE_DE_TRAVAIL,SECTEUR,DATE_D_EXPIRATION_DE_L_OFFRE
0,(Senior) Country Officer,"Santo Domingo,Dominican Republic",Investment,4/1/2024
1,Accounting Officer,"Washington, DC,United States",Finance & Accounting,4/5/2024
2,Associate Investment Officer,"Nairobi,Kenya",Investment,3/27/2024
3,Associate Investment Officer,"Belgrade,Serbia",Investment,4/10/2024
4,Associate Investment Officer,"Beijing,China",Investment,4/5/2024
5,Associate Investment Officer / Investment Officer,"Rabat,Morocco",Investment,4/4/2024
6,Associate/Operations Officer,"Vienna,Austria",Operations,3/29/2024
7,Business Process Analyst,"Chennai,India",Finance & Accounting,4/5/2024
8,Business Process Associate,"Chennai,India",Finance & Accounting,4/5/2024
9,Climate Change Specialist (Markets),"Washington, DC,United States",Climate Change,3/26/2024


# yop_l_frii

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

def yop_l_frii():
    # List of URLs
    urls = [
        f"https://yop.l-frii.com/offres-demplois/{category}/" for category in range(3276)
        # Add more URLs as needed
    ]

    job_data = []

    for url in urls:
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
            soup = BeautifulSoup(response.content, 'html.parser')

            job_articles = soup.find_all('article', class_='type-emploi')

            for article in job_articles:
                job_title_element = article.find('h2', class_='elementor-heading-title')
                job_title = job_title_element.text.strip() if job_title_element else None

                job_link = article.find('a', href=True)['href']

                job_image_element = article.find('img', class_='attachment-large')
                job_image = job_image_element['src'] if job_image_element else None

                job_data.append({
                    "Job Title": job_title,
                    "Job Link": job_link,
                    "Job Image": job_image,
                    "Source URL": url
                })
        except requests.exceptions.RequestException as e:
            print(f"An error occurred while fetching data from {url}: {e}")

    df_yop_l_frii = pd.DataFrame(job_data)

    # Définition de l'équivalence entre les variables
    equivalences = {
        "Job Title": "INTITULE_DU_POSTE",
        "Job Link": "URL_DU_POSTE",
        "Job Image": None,
        "Source URL": None
    }

    # Fonction pour renommer les colonnes du DataFrame en conservant les colonnes sans équivalence
    def renommer_colonnes(df, equivalences):
        colonnes_renommees = {ancien_nom: nouvel_nom for ancien_nom, nouvel_nom in equivalences.items() if
                              nouvel_nom is not None}
        df_renomme = df.rename(columns=colonnes_renommees)
        return df_renomme

    # Renommer les colonnes du DataFrame
    df_yop_l_frii = renommer_colonnes(df_yop_l_frii, equivalences)

    # Afficher le DataFrame avec les colonnes renommées
    return df_yop_l_frii

# Appel de la fonction principale
yop_l_frii=yop_l_frii()
yop_l_frii

,INTITULE_DU_POSTE,URL_DU_POSTE,Job Image,Source URL
0,La Banque mondiale recrute pour ce poste (27 M...,https://yop.l-frii.com/emploi/la-banque-mondia...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/
1,EXPERTISE FRANCE recrute pour ces 02 postes (2...,https://yop.l-frii.com/emploi/expertise-france...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/
2,L’organisation humanitaire ACF recrute pour ce...,https://yop.l-frii.com/emploi/lorganisation-hu...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/
3,SOCIETE GENERALE recrute pour ce poste (27 Mar...,https://yop.l-frii.com/emploi/societe-generale...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/
4,Le Groupe COLAS recrute pour ces 05 postes (26...,https://yop.l-frii.com/emploi/le-groupe-colas-...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/
...,...,...,...,...
39307,Le Conseil Danois pour les Réfugiés (DRC) recr...,https://yop.l-frii.com/emploi/le-conseil-danoi...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/
39308,L’ONG Educo recrute pour ce poste (23 Décembre...,https://yop.l-frii.com/emploi/long-educo-recru...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/
39309,Maersk line recrute pour ce poste (22 Décembre...,https://yop.l-frii.com/emploi/maersk-line-recr...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/
39310,Clinton Health Access Initiative (CHAI) recrut...,https://yop.l-frii.com/emploi/clinton-health-a...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/


In [19]:
yop_l_frii['INTITULE_DU_POSTE'] = yop_l_frii['INTITULE_DU_POSTE'].str.replace(r'[()]', '')

# Utilisation de str.extract() pour extraire les parties de la chaîne
yop_l_frii['DATE_DE_DEBUT_DE_L_OFFRE'] = yop_l_frii['INTITULE_DU_POSTE'].str.extract(r'(\d+\s\w+\s\d+)$')



# Remplacer la partie extraite par une chaîne vide pour obtenir INTITULE_DU_POSTE
yop_l_frii['INTITULE_DU_POSTE'] = yop_l_frii['INTITULE_DU_POSTE'].str.replace(r'(\d+\s\w+\s\d+)$', '')
yop_l_frii['NOMBRE_DE_POSTE_DE_TRAVAIL'] = yop_l_frii['INTITULE_DU_POSTE'].str.extract(r'(\d+)\s*postes')

# Affichage du DataFrame résultant
yop_l_frii

,INTITULE_DU_POSTE,URL_DU_POSTE,Job Image,Source URL,DATE_DE_DEBUT_DE_L_OFFRE,NOMBRE_DE_POSTE_DE_TRAVAIL
0,La Banque mondiale recrute pour ce poste,https://yop.l-frii.com/emploi/la-banque-mondia...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/,27 Mars 2024,NaN
1,EXPERTISE FRANCE recrute pour ces 02 postes,https://yop.l-frii.com/emploi/expertise-france...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/,27 Mars 2024,02
2,L’organisation humanitaire ACF recrute pour ce...,https://yop.l-frii.com/emploi/lorganisation-hu...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/,27 Mars 2024,NaN
3,SOCIETE GENERALE recrute pour ce poste,https://yop.l-frii.com/emploi/societe-generale...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/,27 Mars 2024,NaN
4,Le Groupe COLAS recrute pour ces 05 postes,https://yop.l-frii.com/emploi/le-groupe-colas-...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/,26 Mars 2024,05
...,...,...,...,...,...,...
39307,Le Conseil Danois pour les Réfugiés DRC recrut...,https://yop.l-frii.com/emploi/le-conseil-danoi...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/,23 Décembre 2021,2
39308,L’ONG Educo recrute pour ce poste,https://yop.l-frii.com/emploi/long-educo-recru...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/,23 Décembre 2021,NaN
39309,Maersk line recrute pour ce poste,https://yop.l-frii.com/emploi/maersk-line-recr...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/,22 Décembre 2021,NaN
39310,Clinton Health Access Initiative CHAI recrute ...,https://yop.l-frii.com/emploi/clinton-health-a...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/,21 Décembre 2021,NaN


In [ ]:
yop_l_frii['INTITULE_DU_POSTE'].str.extract(r'(\d+)\s*postes')

In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_job_information(url):
    try:
        # Envoyer une requête GET à l'URL
        response = requests.get(url)
        response.raise_for_status()  # Lever une exception en cas d'erreur HTTP
        # Utiliser BeautifulSoup pour analyser le contenu HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Rechercher les éléments contenant les informations sur l'emploi
        job_info_elements = soup.find_all('div', class_='elementor-widget-container')

        # Initialiser un dictionnaire pour stocker les informations
        job_info = {'URL_DU_POSTE': url}

        # Parcourir les éléments et extraire les informations
        for element in job_info_elements:
            # Trouver le titre de l'emploi
            title_element = element.find('h2', class_='elementor-heading-title')
            if title_element:
                job_info['Titre du Poste'] = title_element.text.strip()

            # Trouver les autres informations sur l'emploi
            other_info_elements = element.find_all('h2', class_='elementor-heading-title')
            for info_element in other_info_elements:
                info_text = info_element.text.strip()
                # Vérifier chaque élément d'information et l'ajouter au dictionnaire
                if 'Niveau Requis' in info_text:
                    job_info['Niveau Requis'] = info_text.split(':')[-1].strip()
                elif 'Année d\'Expérience Requise' in info_text:
                    job_info['Année d\'Expérience Requise'] = info_text.split(':')[-1].strip()
                elif 'Lieu du Travail' in info_text:
                    job_info['Lieu du Travail'] = info_text.split(':')[-1].strip()
                elif 'Date de Soumission' in info_text:
                    job_info['Date de Soumission'] = info_text.split(':')[-1].strip()

        return job_info
    except Exception as e:
        print(f"Erreur lors de l'extraction des informations pour l'URL {url}: {e}")
        return {'URL_DU_POSTE': url}

def extract_job_info_from_urls(urls):
    # Initialiser une liste pour stocker les informations sur les emplois
    job_info_list = []
    # Parcourir les URLs et extraire les informations sur les emplois
    for url in urls:
        job_info = extract_job_information(url)
        job_info_list.append(job_info)
    # Créer un DataFrame à partir de la liste des informations sur les emplois
    df = pd.DataFrame(job_info_list)
    return df

# Liste des URLs
urls = list(yop_l_frii["URL_DU_POSTE"])

# Extraire les informations sur les emplois à partir des URLs
df = extract_job_info_from_urls(urls)

# Afficher le DataFrame
df

Erreur lors de l'extraction des informations pour l'URL https://yop.l-frii.com/emploi/long-internationale-alima-recrute-pour-ces-postes-25-mars-2024/: 404 Client Error: Not Found for url: https://yop.l-frii.com/emploi/long-internationale-alima-recrute-pour-ces-postes-25-mars-2024/
Erreur lors de l'extraction des informations pour l'URL https://yop.l-frii.com/emploi/long-britannique-tearfund-recrute-pour-ce-poste-14-janvier-2024/: 504 Server Error: Gateway Time-out for url: https://yop.l-frii.com/emploi/long-britannique-tearfund-recrute-pour-ce-poste-14-janvier-2024/


,URL_DU_POSTE,Titre du Poste,Niveau Requis,Année d'Expérience Requise,Lieu du Travail,Date de Soumission
0,https://yop.l-frii.com/emploi/la-banque-mondia...,2023 © L-FRII MEDIA,Diplôme universitaire supérieur,5 ans,Sénégal,15/04/2024
1,https://yop.l-frii.com/emploi/expertise-france...,2023 © L-FRII MEDIA,NaN,NaN,Côte d'Ivoire/Cameroun,NaN
2,https://yop.l-frii.com/emploi/lorganisation-hu...,2023 © L-FRII MEDIA,"Licence, Master",3 ans,Nigéria,02/04/2024
3,https://yop.l-frii.com/emploi/societe-generale...,2023 © L-FRII MEDIA,Licence,NaN,Ghana,NaN
4,https://yop.l-frii.com/emploi/le-groupe-colas-...,2023 © L-FRII MEDIA,NaN,NaN,Côte d'Ivoire,NaN
...,...,...,...,...,...,...
39307,https://yop.l-frii.com/emploi/le-conseil-danoi...,2023 © L-FRII MEDIA,NaN,NaN,Niger,NaN
39308,https://yop.l-frii.com/emploi/long-educo-recru...,2023 © L-FRII MEDIA,BAC + 3,3 ans,Niger,05/01/2022
39309,https://yop.l-frii.com/emploi/maersk-line-recr...,2023 © L-FRII MEDIA,NaN,3 à 5 ans,"Abidjan, Côte d'Ivoire",31/12/2021
39310,https://yop.l-frii.com/emploi/clinton-health-a...,2023 © L-FRII MEDIA,Master,"4 ans, 5 ans, 6 ans",Bénin,NaN


In [16]:
df.head(2)

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,Niveau,Experience_lettre,url,SITE_WEB_DE_L_ENTREPRISE,RAISON_SOCIALE_DE_L_ENTREPRISE,BRANCHE_D_ACTIVITE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,NOMBRE_DE_POSTES_A_POURVOIR,Niveau de poste,Niveau d'étude (diplome),TYPE_DE_CONTRAT_DU_POSTE,Texte_fourni,VILLE_DU_POSTE_DE_TRAVAIL
0,Livreurs Moto,Entreprise anonyme,Expérimenté,04 Janvier,Confirmé / Expérimenté,Sans expérience,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Entreprise anonyme,Services,Côte d'ivoire,03 Avril,40,Débutant / Junior| Stagiaire / Etudiant| Confi...,Niveau secondaire| Niveau terminal| Baccalauré...,Mission,SPPCI Offre d'emploiLivreurs Moto (H/F)Vous êt...,Confirmé
1,Gestionnaire des Ressources Humaines,None,Responsable département,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,Services,"Abidjan, Côte d'ivoire",22 Juin,NaN,Manager / Responsable département,NaN,NaN,Entreprise spécialisée dans l’ingénierie mécan...,Manager


In [25]:
# Utilisation de str.extract() pour extraire les nombres min et max
extracted_data = df['Année d\'Expérience Requise'].str.extract(r'(\d+)\s*ans,\s*(\d+)\s*ans?')  # Extraction des années min et max

# Renommer les colonnes extraites
extracted_data.columns = ['annee_min', 'annee_max']  # Renommage des colonnes extraites

# Remplir les valeurs manquantes dans annee_min avec les valeurs extraites de Année d'Expérience Requise
extracted_data['annee_min'].fillna(df["Année d'Expérience Requise"], inplace=True)  # Remplissage des valeurs manquantes dans la colonne 'annee_min' avec les valeurs d'origine
extracted_data['annee_max'].fillna(df["Année d'Expérience Requise"], inplace=True)  # Remplissage des valeurs manquantes dans la colonne 'annee_max' avec les valeurs d'origine

# Extraire uniquement les chiffres de chaque colonne
extracted_data['annee_min'] = extracted_data['annee_min'].str.extract(r"([0-9,]+)")  # Extraction des chiffres de la colonne 'annee_min'
extracted_data['annee_max'] = extracted_data['annee_max'].str.extract(r"([0-9,]+)")  # Extraction des chiffres de la colonne 'annee_max'

# Ajouter les colonnes extraites au DataFrame d'origine
df = pd.concat([df, extracted_data], axis=1)  # Ajout des colonnes extraites au DataFrame d'origine

# Affichage du DataFrame résultant
# Diviser la colonne 'Lieu du Travail' en ville et pays
df[['LIEU_DU_POSTE_DE_TRAVAIL', 'PAYS_DU_POSTE_DE_TRAVAIL']] = df['Lieu du Travail'].str.split(', ', n=1, expand=True)

for i in range(len(df["PAYS_DU_POSTE_DE_TRAVAIL"])):
    if df["PAYS_DU_POSTE_DE_TRAVAIL"][i] is None :
        df["PAYS_DU_POSTE_DE_TRAVAIL"][i] = df["LIEU_DU_POSTE_DE_TRAVAIL"][i]
df  # Affichage du DataFrame résultant

,URL_DU_POSTE,Titre du Poste,Niveau Requis,Année d'Expérience Requise,Lieu du Travail,Date de Soumission,annee_min,annee_max,LIEU_DU_POSTE_DE_TRAVAIL,PAYS_DU_POSTE_DE_TRAVAIL
0,https://yop.l-frii.com/emploi/la-banque-mondia...,2023 © L-FRII MEDIA,Diplôme universitaire supérieur,5 ans,Sénégal,15/04/2024,5,5,Sénégal,Sénégal
1,https://yop.l-frii.com/emploi/expertise-france...,2023 © L-FRII MEDIA,NaN,NaN,Côte d'Ivoire/Cameroun,NaN,NaN,NaN,Côte d'Ivoire/Cameroun,Côte d'Ivoire/Cameroun
2,https://yop.l-frii.com/emploi/lorganisation-hu...,2023 © L-FRII MEDIA,"Licence, Master",3 ans,Nigéria,02/04/2024,3,3,Nigéria,Nigéria
3,https://yop.l-frii.com/emploi/societe-generale...,2023 © L-FRII MEDIA,Licence,NaN,Ghana,NaN,NaN,NaN,Ghana,Ghana
4,https://yop.l-frii.com/emploi/le-groupe-colas-...,2023 © L-FRII MEDIA,NaN,NaN,Côte d'Ivoire,NaN,NaN,NaN,Côte d'Ivoire,Côte d'Ivoire
...,...,...,...,...,...,...,...,...,...,...
39307,https://yop.l-frii.com/emploi/le-conseil-danoi...,2023 © L-FRII MEDIA,NaN,NaN,Niger,NaN,NaN,NaN,Niger,Niger
39308,https://yop.l-frii.com/emploi/long-educo-recru...,2023 © L-FRII MEDIA,BAC + 3,3 ans,Niger,05/01/2022,3,3,Niger,Niger
39309,https://yop.l-frii.com/emploi/maersk-line-recr...,2023 © L-FRII MEDIA,NaN,3 à 5 ans,"Abidjan, Côte d'Ivoire",31/12/2021,3,3,Abidjan,Côte d'Ivoire
39310,https://yop.l-frii.com/emploi/clinton-health-a...,2023 © L-FRII MEDIA,Master,"4 ans, 5 ans, 6 ans",Bénin,NaN,4,5,Bénin,Bénin


In [23]:
df

,INTITULE_DU_POSTE,Entreprise,PAYS_DU_POSTE_DE_TRAVAIL,DATE_DE_DEBUT_DE_L_OFFRE,Niveau,Experience_lettre,url,SITE_WEB_DE_L_ENTREPRISE,RAISON_SOCIALE_DE_L_ENTREPRISE,BRANCHE_D_ACTIVITE,LIEU_DU_POSTE_DE_TRAVAIL,DATE_D_EXPIRATION_DE_L_OFFRE,NOMBRE_DE_POSTES_A_POURVOIR,Niveau de poste,Niveau d'étude (diplome),TYPE_DE_CONTRAT_DU_POSTE,Texte_fourni,VILLE_DU_POSTE_DE_TRAVAIL
0,Livreurs Moto,Entreprise anonyme,Expérimenté,04 Janvier,Confirmé / Expérimenté,Sans expérience,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,Entreprise anonyme,Services,Côte d'ivoire,03 Avril,40,Débutant / Junior| Stagiaire / Etudiant| Confi...,Niveau secondaire| Niveau terminal| Baccalauré...,Mission,SPPCI Offre d'emploiLivreurs Moto (H/F)Vous êt...,Confirmé
1,Gestionnaire des Ressources Humaines,None,Responsable département,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,Services,"Abidjan, Côte d'ivoire",22 Juin,NaN,Manager / Responsable département,NaN,NaN,Entreprise spécialisée dans l’ingénierie mécan...,Manager
2,Gestionnaire des Ressources Humaines,None,Responsable département,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,Services,"Abidjan, Côte d'ivoire",22 Juin,NaN,Manager / Responsable département,NaN,NaN,Entreprise spécialisée dans l’ingénierie mécan...,Manager
3,Gestionnaire des Ressources Humaines,None,Responsable département,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,Services,"Abidjan, Côte d'ivoire",22 Juin,NaN,Manager / Responsable département,NaN,NaN,Entreprise spécialisée dans l’ingénierie mécan...,Manager
4,Gestionnaire des Ressources Humaines,None,Responsable département,24 Mars,Manager / Responsable département,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,Services,"Abidjan, Côte d'ivoire",22 Juin,NaN,Manager / Responsable département,NaN,NaN,Entreprise spécialisée dans l’ingénierie mécan...,Manager
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,Maintenance Officer,OLAM FOOD Ingrédients (OFI),Expérimenté,31 Janvier,Confirmé / Expérimenté,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"Distribution, Commerce","Bouake, Côte d'ivoire",30 Avril,01,Confirmé / Expérimenté,Formation Professionnelle,CDI,Position within the structure :Position based ...,Confirmé
782,Maintenance Officer,OLAM FOOD Ingrédients (OFI),Expérimenté,31 Janvier,Confirmé / Expérimenté,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"Distribution, Commerce","Bouake, Côte d'ivoire",30 Avril,01,Confirmé / Expérimenté,Formation Professionnelle,CDI,Position within the structure :Position based ...,Confirmé
783,Maintenance Officer,OLAM FOOD Ingrédients (OFI),Expérimenté,31 Janvier,Confirmé / Expérimenté,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"Distribution, Commerce","Bouake, Côte d'ivoire",30 Avril,01,Confirmé / Expérimenté,Formation Professionnelle,CDI,Position within the structure :Position based ...,Confirmé
784,Production in Charge,OLAM FOOD Ingrédients (OFI),Expérimenté,06 Février,Confirmé / Expérimenté,6 à 10 ans,https://www.novojob.com/cote-d-ivoire/offres-d...,https://www.novojob.com/cote-d-ivoire/offres-d...,NaN,"Distribution, Commerce","Bouake, Côte d'ivoire",06 Mai,01,Confirmé / Expérimenté,"Master 1, Licence Bac + 4",CDI,Présentation de l’entreprise \nOLAM FOOD ingre...,Confirmé


In [26]:
yop_l_frii_total = pd.merge(yop_l_frii,df, on='URL_DU_POSTE')
yop_l_frii_total = yop_l_frii_total.drop_duplicates()
yop_l_frii_total.reset_index(drop=True, inplace=True)
yop_l_frii_total

,INTITULE_DU_POSTE,URL_DU_POSTE,Job Image,Source URL,DATE_DE_DEBUT_DE_L_OFFRE,NOMBRE_DE_POSTE_DE_TRAVAIL,Titre du Poste,Niveau Requis,Année d'Expérience Requise,Lieu du Travail,Date de Soumission,annee_min,annee_max,LIEU_DU_POSTE_DE_TRAVAIL,PAYS_DU_POSTE_DE_TRAVAIL
0,La Banque mondiale recrute pour ce poste,https://yop.l-frii.com/emploi/la-banque-mondia...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/,27 Mars 2024,NaN,2023 © L-FRII MEDIA,Diplôme universitaire supérieur,5 ans,Sénégal,15/04/2024,5,5,Sénégal,Sénégal
1,EXPERTISE FRANCE recrute pour ces 02 postes,https://yop.l-frii.com/emploi/expertise-france...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/,27 Mars 2024,02,2023 © L-FRII MEDIA,NaN,NaN,Côte d'Ivoire/Cameroun,NaN,NaN,NaN,Côte d'Ivoire/Cameroun,Côte d'Ivoire/Cameroun
2,L’organisation humanitaire ACF recrute pour ce...,https://yop.l-frii.com/emploi/lorganisation-hu...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/,27 Mars 2024,NaN,2023 © L-FRII MEDIA,"Licence, Master",3 ans,Nigéria,02/04/2024,3,3,Nigéria,Nigéria
3,SOCIETE GENERALE recrute pour ce poste,https://yop.l-frii.com/emploi/societe-generale...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/,27 Mars 2024,NaN,2023 © L-FRII MEDIA,Licence,NaN,Ghana,NaN,NaN,NaN,Ghana,Ghana
4,Le Groupe COLAS recrute pour ces 05 postes,https://yop.l-frii.com/emploi/le-groupe-colas-...,https://yop.l-frii.com/wp-content/uploads/2024...,https://yop.l-frii.com/offres-demplois/0/,26 Mars 2024,05,2023 © L-FRII MEDIA,NaN,NaN,Côte d'Ivoire,NaN,NaN,NaN,Côte d'Ivoire,Côte d'Ivoire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39307,Le Conseil Danois pour les Réfugiés DRC recrut...,https://yop.l-frii.com/emploi/le-conseil-danoi...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/,23 Décembre 2021,2,2023 © L-FRII MEDIA,NaN,NaN,Niger,NaN,NaN,NaN,Niger,Niger
39308,L’ONG Educo recrute pour ce poste,https://yop.l-frii.com/emploi/long-educo-recru...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/,23 Décembre 2021,NaN,2023 © L-FRII MEDIA,BAC + 3,3 ans,Niger,05/01/2022,3,3,Niger,Niger
39309,Maersk line recrute pour ce poste,https://yop.l-frii.com/emploi/maersk-line-recr...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/,22 Décembre 2021,NaN,2023 © L-FRII MEDIA,NaN,3 à 5 ans,"Abidjan, Côte d'Ivoire",31/12/2021,3,3,Abidjan,Côte d'Ivoire
39310,Clinton Health Access Initiative CHAI recrute ...,https://yop.l-frii.com/emploi/clinton-health-a...,https://yop.l-frii.com/wp-content/uploads/2021...,https://yop.l-frii.com/offres-demplois/3275/,21 Décembre 2021,NaN,2023 © L-FRII MEDIA,Master,"4 ans, 5 ans, 6 ans",Bénin,NaN,4,5,Bénin,Bénin


# emploi_ci

In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

from selenium import webdriver

def emploi_ci():
    def extract_text(element, tag_name=None):
        tag = element.find(tag_name)
        return tag.text.strip() if tag else ""

    def clean_text(text):
        return text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')

    def scrape_emploi_ci(url):
        try:
            response = requests.get(url, timeout=500)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion à {url} : {e}")
            return pd.DataFrame()

        soup = BeautifulSoup(response.text, 'html.parser')

        job_description_wrappers = soup.find_all('div', class_='job-description-wrapper')

        data_list = []

        for wrapper in job_description_wrappers:
            h5_tag = wrapper.find('h5')
            poste = extract_text(h5_tag, 'a')

            job_recruiter_tag = wrapper.find('p', class_='job-recruiter')
            date_and_company = job_recruiter_tag.text.strip().split('|')
            date = date_and_company[0].strip() if date_and_company else ""
            entreprise = extract_text(job_recruiter_tag, 'a')

            description_tag = wrapper.find('div', class_='search-description')
            description = clean_text(description_tag.text.strip()) if description_tag else ""

            region_tag = wrapper.find('p', text='Région de :')
            region = extract_text(region_tag) if region_tag else ""

            data_list.append({
                'Poste': poste,
                'Entreprise': entreprise,
                'Date': date,
                'Description': description,
                'Région': region,
                'URL' : url
            })

        df = pd.DataFrame(data_list)
        return df

    # Liste des liens
    categories = ["31", "1127", "29", "37", "1115", "30", "1115", "32", "33", "34", "35", "36", "37", "39", "38", "40", "525", "41", "28"]
    #categories=["31"]
    # Liste d'URLs générées
    urls = ["https://www.emploi.ci/recherche-jobs-cote-ivoire/?f%5B0%5D=im_field_offre_metiers%3A{}".format(category) for category in categories]

    # Créer un DataFrame à partir des liens
    df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)


    from requests.exceptions import ChunkedEncodingError, ConnectionError, ReadTimeout

    # Liste des liens

    # En-tête pour éviter d'être bloqué
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
    options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
    chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
    options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
    options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
    driver = webdriver.Chrome(options=options)

    # Liste pour stocker les détails de chaque emploi
    all_job_details = []

    # Parcourir les liens
    for url in list(df["URL"]):
        req = requests.get(url, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

        offres = soup.find_all('div', class_="job-description-wrapper")

        # Parcourir les offres d'emploi sur la page principale
        for offre in offres:
            # Trouver la balise <h4> dans la structure HTML pour extraire le lien
            offre_link_tag = offre.find('h5')

            # Vérifier si la balise <h4> a été trouvée
            if offre_link_tag:
                # Extraire le lien de l'attribut 'href'
                offre_link = offre_link_tag.find('a')['href']
                all_job_details.append({'Offre_Link': "https://www.emploi.ci"+offre_link, 'URL' :url})

    # Fermer le pilote Selenium à la fin
    driver.quit()

    # Concaténer tous les détails des emplois en un seul DataFrame
    if all_job_details:
        all_job_details_df = pd.DataFrame(all_job_details)
        # Afficher le DataFrame
        #print(all_job_details_df)
    else:
        print("Aucun détail d'offre d'emploi trouvé.")

    # Fusionner les deux DataFrames sur la colonne 'URL'
    emploi_df = pd.merge(df, all_job_details_df, on='URL')
    emploi_df = emploi_df.drop_duplicates()
    
    #df["URL"]=list(all_job_details_df["URL"])
    #df["Offre_Link"]=list(all_job_details_df["Offre_Link"])

    # Fonction pour extraire les informations d'une page
    def extract_information(url):
        try:
            response = requests.get(url, timeout=120)  # Augmentation du délai à 20 secondes
            response.raise_for_status()
            response.encoding = 'utf-8'

            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraction des informations sur l'entreprise
            company_info = soup.select_one('.job-ad-company')
            entreprise = {
                "Offre_Link" : url,
                'Nom': company_info.select_one('.company-title a').text.strip() if company_info and company_info.select_one('.company-title a') else None,
                'Secteur d´activité': ', '.join(item.text.strip() for item in company_info.select('.sector-title .field-item')) if company_info and company_info.select('.sector-title .field-item') else None,
                'Description de l\'entreprise': soup.select_one('.job-ad-company-description label + *').text.strip() if soup.select_one('.job-ad-company-description label + *') else None
            }

            # Extraction des informations sur l'annonce
            annonce_info = soup.select_one('.job-ad-details')
            annonce = {
                'Poste': soup.select_one('.ad-ss-title').text.strip() if soup.select_one('.ad-ss-title') else None,
                'Missions': [li.text.strip() for li in soup.select('.content ul.missions li')] if soup.select('.content ul.missions') else None,
                'Profil recherché': [li.text.strip() for li in soup.select('.content ul.profil li')] if soup.select('.content ul.profil') else None,
                'Métier': soup.select_one('.job-ad-criteria .field-name-field-offre-metiers .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-metiers .field-item') else None,
                'Secteur d´activité (de l\'annonce)': soup.select_one('.job-ad-criteria .field-name-field-offre-secteur .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-secteur .field-item') else None,
                'Type de contrat': soup.select_one('.job-ad-criteria .field-name-field-offre-contrat-type .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-contrat-type .field-item') else None,
                'Région': soup.select_one('.job-ad-criteria .field-name-field-offre-region .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-region .field-item') else None,
                'Ville': soup.select_one('.job-ad-criteria .field-name-field-offre-ville .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-ville .field-item') else None,
                'Niveau d\'expérience': soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-experience .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-experience .field-item') else None,
                'Niveau d\'études': soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-etude .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-etude .field-item') else None,
                'Compétences clés': [li.text.strip() for li in soup.select('.job-ad-criteria .field-name-field-offre-tags .field-item')] if soup.select('.job-ad-criteria .field-name-field-offre-tags .field-item') else None,
                'Nombre de poste(s)': soup.select_one('.job-ad-criteria td:contains("Nombre de poste(s) :") + td').text.strip() if soup.select_one('.job-ad-criteria td:contains("Nombre de poste(s) :") + td') else None,
            }

            return {'entreprise': entreprise, 'annonce': annonce}

        except (ConnectionError, ReadTimeout, ChunkedEncodingError) as e:
            print(f"Erreur lors de la requête {url}: {e}")
            # Relancer la requête
            entreprise = {
                "Offre_Link" : url,
                'Nom': "",
                'Secteur d´activité': "",
                'Description de l\'entreprise':""}
            annonce = {'Poste':"",
                       'Missions': "",
                       'Profil recherché':"",
                       'Métier':"",
                       'Secteur d´activité (de l\'annonce)':"",
                       'Type de contrat':"",
                       'Région': "",
                       'Ville':"",
                       'Niveau d\'expérience':"",
                       'Niveau d\'études':"",
                       'Compétences clés':"",
                       'Nombre de poste(s)':""}


            return {'entreprise': entreprise, 'annonce': annonce}

    # Liste des URLs
    urls = list(emploi_df['Offre_Link'])

    # Initialisation d'une liste pour stocker les DataFrames
    df_list = []

    # Boucle à travers chaque URL
    for url in urls:
        data = extract_information(url)

        # Si la requête a échoué, passez à l'URL suivante
        if data is None:
            continue

        # Création du DataFrame pour chaque URL
        df = pd.DataFrame([data['entreprise'] | data['annonce']])

        # Ajout du DataFrame à la liste
        df_list.append(df)

    # Concaténation des DataFrames de chaque URL
    result_df = pd.concat(df_list, ignore_index=True)
    # Ajouter les listes existantes en tant que colonnes au DataFrame
    result_df = pd.merge(result_df, emploi_df, on='Offre_Link')
    result_df = emploi_df.drop_duplicates()


# Réorganiser les colonnes selon vos besoins
#Poste 	Entreprise 	Date 	Description 	Région 	URL 	Offre_Link

    return result_df

# Appel de la fonction pour obtenir le DataFrame
emploi_df=emploi_ci()
emploi_df.reset_index(drop=True, inplace=True)
emploi_df

C:\Users\Dell\anaconda3\lib\site-packages\soupsieve\css_parser.py:876: FutureWarning:

The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.



Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/consultant-strategie-it-business-analyst-amplitude-1395398: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/consultant-technique-it-microsoft-specialiste-infrastructure-syst-1395401: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/consultant-technique-it-microsoft-specialiste-infrastructure-syst-1395401 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000211FAB380A0>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/stagiaire-designer-telecom-hf-1390769: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/stagiaire-designer-telecom-hf-1390769 (Caused by ConnectT

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/consultant-securite-informatique-amplitude-abidjan-cote-ivoire-1395364: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/consultant-securite-informatique-amplitude-abidjan-cote-ivoire-1395364 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000211F7E9A0B0>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/consultant-maintenance-support-technique-amplitude-1395367: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/consultant-maintenance-support-technique-amplitude-1395367 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000211F7E9B610>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.c

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/it-architect-1393749: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/it-architect-1393749 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000211F36DFEB0>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/analyste-donnees-decisionnelles-bi-champion-okr-1403295: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/analyste-donnees-decisionnelles-bi-champion-okr-1403295 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000211F36DEE30>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/country-manager-in-technology-1399466: HTTPSConnectionPool(host='www.emploi.ci'

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/dotnet-developer-mf-1386341: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/dotnet-developer-mf-1386341 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000211FBD8B760>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/ios-developer-mf-1386342: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/ios-developer-mf-1386342 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000211FBD8B130>, 'Connection to www.emploi.ci timed out. (connect timeout=120)'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/java-developer-mf-1386343: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-co

...

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/controleur-gestion-groupe-1392726: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/controleur-gestion-groupe-1392726 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000211FAE3D7E0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/directeur-technique-1392861: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/directeur-technique-1392861 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000211FAE3E770>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/directeur-administratif-financier-bilingue-1389952: HTTPSConnectionPool(host='www.emploi.ci

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/commercial-lubrifiants-reseau-distributeurs-hf-1387742: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/commercial-lubrifiants-reseau-distributeurs-hf-1387742 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000211FA5768C0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/commercial-energies-transition-hf-1387745: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/commercial-energies-transition-hf-1387745 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000211FAFD4820>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/commercial-zone-14051

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/directeur-technique-1389411: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/directeur-technique-1389411 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000211FA6D0AC0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/technicienne-son-freelance-abidjan-urgent-1402257: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/technicienne-son-freelance-abidjan-urgent-1402257 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000211FA6D3730>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/maitre-hotel-1403789: HTTPSConnectionPool(host='www.emploi.

Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/assistante-ressources-humaines-stagiaire-1394518: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/assistante-ressources-humaines-stagiaire-1394518 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000211F3477FD0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/charge-paie-administration-1394808: HTTPSConnectionPool(host='www.emploi.ci', port=443): Max retries exceeded with url: /offre-emploi-cote-ivoire/charge-paie-administration-1394808 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000211F3476CE0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Erreur lors de la requête https://www.emploi.ci/offre-emploi-cote-ivoire/directeur-ressources-humaines-1404100: HTTPSCon

,Poste,Entreprise,Date,Description,Région,URL,Offre_Link
0,Hybrid Noc Security Analyst,ENDEAVOUR MINING,27.03.2024,The Hybrid NOC-Security Analyst is a key role ...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
1,Hybrid Noc Security Analyst,ENDEAVOUR MINING,27.03.2024,The Hybrid NOC-Security Analyst is a key role ...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
2,Hybrid Noc Security Analyst,ENDEAVOUR MINING,27.03.2024,The Hybrid NOC-Security Analyst is a key role ...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
3,Hybrid Noc Security Analyst,ENDEAVOUR MINING,27.03.2024,The Hybrid NOC-Security Analyst is a key role ...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
4,Hybrid Noc Security Analyst,ENDEAVOUR MINING,27.03.2024,The Hybrid NOC-Security Analyst is a key role ...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
...,...,...,...,...,...,...,...
2531,Chauffeur Intérimaire H/F,REIME CÔTE D'IVOIRE,15.02.2024,"Si vous êtes passionné(e) par la conduite, que...",,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
2532,Responsable Service Généraux,UMO INTÉRIM,07.02.2024,SOCIÉTÉNotre client est une société internatio...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
2533,Responsable Service Généraux,UMO INTÉRIM,07.02.2024,SOCIÉTÉNotre client est une société internatio...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
2534,Responsable Service Généraux,UMO INTÉRIM,07.02.2024,SOCIÉTÉNotre client est une société internatio...,,https://www.emploi.ci/recherche-jobs-cote-ivoi...,https://www.emploi.ci/offre-emploi-cote-ivoire...
